## Evaluating Transfer Learning Models

This notebook evaluates the performance of the transfer learning experiment models. The following model evaluations are shown in order:

* Baseline performance of the model which was fully trained on Yelp data.
* Performance of the Amazon small and Amazon large models _before_ transfer learning:
  * Metrics on the test set paired to the model's training set.
  * Metrics on the Yelp test set on which the model _wasn't trained_.
* Performance of the BERT base, Amazon small, and Amazon large models _after_ transfer learning:
  * Metrics on the Yelp test set on which the model was *fine-tuned*:
    * Metrics from fine-tuning 2 layers.
    * Metrics from fine-tuning 4 layers.

For each section the following metrics are shown:

* Accuracy
* Classification report
* Confusion matrix - counts and normalized
* Word count comparison - tp, fp, tn, fn
* Top 10 most confident predictions - tp, fp, tn, fn

In [1]:

import datetime
import os
import pickle
import random
import warnings

import pandas as pd
from sklearn.model_selection import train_test_split

from utils import accuracy, confusion_matrix, get_google_drive_download_url, load_full_datasets, local_save_dir, most_confident_predictions

pd.set_option("display.max_colwidth", 500)

MODEL_DIR = os.path.join(os.path.dirname(os.getcwd()), "models")

## Load the raw datasets

In [2]:
datasets = load_full_datasets()

Loading bilal:
> train=7,200, test=2,000, val=800
Loading yelp:
> train=47,146, test=5,894, val=5,893
Loading amazon_small:
> train=48,000, test=6,000, val=6,000
Loading amazon_large:
> train=200,000, test=25,000, val=25,000


## Model performance before transfer learning

In [3]:
results_dirs = {
    "bilal_full_yelp_finetune": os.path.join(MODEL_DIR, "bilal_full_yelp_finetune", "results", "2022_07_19__17_35_20"),
    "amazon_finetune": os.path.join(MODEL_DIR, "amazon_finetune", "results", "2022_07_19__22_18_29"),
    "amazon_finetune_LARGE": os.path.join(MODEL_DIR, "amazon_finetune_LARGE", "results", "2022_07_21__10_46_09"),
}

### Baseline model trained on yelp

Performance of the baseline model when fully trained on the Yelp dataset of ~50k records.

#### Same-domain test set

In [4]:
model_name = "bilal_full_yelp_finetune"
results_fp = results_dirs[model_name]

preds_path = os.path.join(results_fp, "test_predictions.csv")
preds = pd.read_csv(preds_path, encoding="utf-8")
test_set = datasets["yelp"]["test"][["text", "label"]]
preds["text"] = test_set["text"]
preds["label"] = test_set["label"]
preds["num_words"] = [len(t.split()) for t in preds["text"]]
preds.head()

,pred_prob_0,pred_prob_1,yhat,y,category,text,label,num_words
0,-2.729194,2.748902,1,1,tp,"There is a small, evil part of me who really wants to give Charlie's 1 star and a terrible review just so you won't go there and find out about it's goodness. It isn't really big enough to handle the over-capacity of being popular among more folks then the few who are ""in the know"". \n\nBut then I realize that it's not fair, Charlie's is awesome and still kinda hard to find and if you make the effort to get yourself there then you deserve to experience the aforementioned goodness.\n\nCharlie...",1,270
1,1.516319,-1.132340,0,0,tn,Really great the whole experience was amazing had many people refer me to this place and it was worth it all,0,21
2,-0.226882,0.352117,1,0,fp,"One of the few, if not the only restaurant I've been to more than 10 times in Philly. Just amazing!\n\nYou can't say enough good things about the Tahini Shake. They're all great, although my favorites are the coconut and the mint chocolate chip. \nThe fries are perfectly crunchy too and the seasoning is great. \nThe staff are so incredibly nice. \nThere is a small shelf of grocery type items you can buy if you would like as well.\n\nCons:\nIt closes at 7PM. \nSometimes not enough seating. \n...",0,90
3,0.357275,-0.090970,0,0,tn,The lark is a local favorite and it's on every list of top restaurants in Santa Barbara. We really wanted to love this place and had high expectations based on what we read. \n\nThe ambiance is very warm and welcoming with a winding brick outdoor patio lined with iron fireplaces. We went on a Wednesday with reservations and we were glad we did because it was buzzing. \n\nThe catch here for us was the food. We were told that the style was small shared plates and multiple were recommended but ...,0,235
4,-1.343574,1.276339,1,1,tp,"Where to start? This place has so much potential; a great atmosphere, good drink selection and decent food. Where they lack is in execution. \n\nOur most recent experience:\nOrdering a drink should not take 20+ minutes, especially when those drinks are a beer & a glass of wine. Requesting ketchup should not be a 15 minute process after your food comes. Having your server pulled to cover the bar happens, but for no one to pick up their tables is awful customer service; attempting to make up f...",1,148


##### Accuracy

In [5]:
accuracy(preds)

0.7555140821174076

##### Classification report

In [6]:
with open(os.path.join(results_fp, "classification_report.txt")) as report_file:
    print(report_file.read())

              precision    recall  f1-score   support

           0       0.77      0.74      0.75      2992
           1       0.74      0.77      0.76      2902

    accuracy                           0.76      5894
   macro avg       0.76      0.76      0.76      5894
weighted avg       0.76      0.76      0.76      5894




##### Confusion matrix - raw & normalized

In [7]:
confusion_matrix(preds, False)

,Pred = False,Pred = True
,,
Actual = False,2213,779
Actual = True,662,2240


In [8]:
confusion_matrix(preds, True)

,Pred = False,Pred = True
,,
Actual = False,0.375467,0.132168
Actual = True,0.112318,0.380048


##### Word count comparison

In [9]:
preds.groupby(["category"])["num_words"].mean()

category
fn    103.031722
fp    136.177150
tn     59.768188
tp    227.988839
Name: num_words, dtype: float64

##### Most confident - true positives

In [10]:
most_confident_predictions(preds, "tp")

,category,text,num_words
0,tp,"Ehhh, I'll give it 3 stars. Could've be better, could've be worse... \n\nFor starters, we were greeted & continued to get great service from our server, Jordan.\n\nSo we start with the ""specialty,"" $12.50 margaritas. Straight ""heartburn"" sour mix. So sad; I thought I was buying a drink that'd give me an excuse to gamble funds at the nearest Craps table. Not exactly. \n\nNext, I ordered the Calamari as my entrée. Holy Slimy Tentacles, Batman! \n*&%$! \n I hovered over my Hubby's $5.00, Hap...",116
1,tp,"I always regret not reading yelp before going to a new restaurant. And today I regretted it. I didn't understand what was going on with the menu - oh, Asian fusion. OK, that's my bad. \n\nI ended up ordering the fried calamari. It was delicious! It was all body, so if you're a little squeamish, don't worry, you won't have tentacles sticking out. You get a lot of calamari for $12. I got to try one of my friend's shrimp dumplings, and though it was good, a little pricey for $8 for 4 dumplings....",152
2,tp,"A new pho place popped up on my radar, so you know I had to check it out.\n\nPho Kang is located in the same building that used to house Pho Orchid. Someone said that they thought it was the same place, with a different name, but I'm not sure. I stopped in for lunch and there was no one else in here. I started with an order of grilled pork spring rolls and Vietnamese coffee and I got a regular bowl of rare beef and brisket pho for my entree.\n\nThe spring rolls were excellent- packed with fr...",170
3,tp,"I went back here to try the thin crust and the beef. The beef was good, though they don't dip it automatically (or even ask) so it kinda missed on the authentic part. The thin crust was great. It probably had as much cheese as the deep dish had. That was pathetic for a deep dish, but solid for thin crust. So overall, it was much much better.\n\nHowever, I had a terrrible experience with the guy who runs this place. For one, he was half an hour late to reopen after lunch. Whatever, no big de...",329
4,tp,I have been coming to Joe Bob's for at least 4 owners now. The current version is in the middle of a remodel (inside and out). There are still a handful of regulars. Gone are the stale smell of beer coating the floors and some of the worst bathrooms in Sparks. I love the new restrooms and I understand why they switched the M & F restrooms. \n\n The menu has changed but still has most of the staple items I can remember. The wings are priced at $2.00 per wing which seems kinda high. I will say...,205
5,tp,"A few weeks ago, I read about the ""Cronut"" craze in NYC. Thinking to myself, I could never, ever, ever wait in a humungous line for a pastry, nor pay $40 to buy one from a Cronut scalper on Craigslist, I figured it was better left to the New Yorkers, and I would never try one. Oh well.\n\nThen, enter the Do'Sant at Manhattanjack.\n\nIt's not a Cronut. It's a Do'Sant. Completely different. \nWhy? Because it's made in NOLA, and it's $3.25.\n\nThe hubby and I each got one.... one cinnamon sugar...",141
6,tp,"Don't believe the hype!!\nHighly Over rated. Should have stuck with my 1st mind. Can't go wrong at Stein's.\nThe fried bologna sandwich is a joke. 2 pieces of bologna tons of lettuce all served between 2 ridiculous pieces of TX toast for $10.50. I guess because they have to pay 8 cooks in the kitchen to make sandwiches.\nAlso, do they really think putting chips on a sandwich is original? \nThis has to be a transplants idea. You can't fool a New Orleanian. Remember, we overstuff sandwiches d...",106
7,tp,"I lived in easy walking distance of Hank's for a year, and though I loved Mardi Gras Zone (just down Port St a couple blocks) and wrote a paean to its wonders, I never really appreciated Hank's, certainly not enough to send it up a sweet review.\n\nBut I get it now. I get you, Hank's. \n\nHank's is precisely on my path to Red's Chinese. So on days when I'm doing the bare minimum of humaning

##### Most confident - true negatives

In [11]:
most_confident_predictions(preds, "tn")

,category,text,num_words
0,tn,"I am usually a regular Moe's customer when it comes to catering. I was very disappointed when I ordered catering a few days in advanced and even called to confirm. I told the restaurant that I would need the food to be delivered by 11:15, they lost my order and didn't have it there until 1:00 p.m.",57
1,tn,"My fiancé and I LOVE harvest!! We come here on our own, have brought friends and talk it up! We even brought our families here to meet after our engagement! Somehow we are lucky enough to ALWAYS get the same waitress, Veronica! She's awesome! She knows what is great, what we would personally like and much more, completely knowledgeable! The food is amazing! Absolutely everything on the menu! The drinks, just as amazing! Highly recommended!! Hurry in to harvest and have a great atmosphere wit...",100
2,tn,This place is online taking online orders and does not post anywhere that this is the only way to order food through them. They let you walk in a stand in line but don't have the courtesy to tell you that it's online orders only. I love chipotle but they've got to hire better workers and have to allow in person orders for folks willing to make the trip to their store.,72
3,tn,"I went to the Magazine St location. I used the online wait list which worked perfectly. I arrived at the time stayed and was seated within a couple of minutes. Had a Bloody Mary and two types of Eggs Benedict. Prompt, courteous service and everything was delicious!",47
4,tn,Came around 10.. ordered food.. never got sauce we asked for.. tried to order a drink around 10:35.. was told we had just missed last call.. places closes at 11.. server walks by at 10:50 request a drink to made.. no problem!! 1 star don't recommend,46
5,tn,"Whoa just based on scone alone. Might be best scone ive ever had (and in nyc had a lot ) . Not dry like most. Put a little jam on it. Pleasant interior, friendly staff. I cant speak to other foods tho",42
6,tn,I ordered a steak hoagie and when it was delivered it was just a regular cheesesteak! If i wanted a cheesesteak I would've just ordered a cheesesteak!!! I'm pretty sure on the receipt it said steak HOAGIE!!! Where was my lettuce tomatoes n onions????? I just don't understand why they would leave that out.!,54
7,tn,Best veg burger in town. Go see eddie at the bar! The cocktail selection is fantastic but he can whip you up anything based on your buds. And did I mention... get the cookies to take home with you!!!,39
8,tn,"Had order delivered to Hotel. Arrived packed well, hot--plenty of extra sauces. I need my Thai fix whenever I work out of town. This was a great fix and will recommend it to my friends when in Nashville, Tennesse.",39
9,tn,Justin took care of us for our friends bachelorette and he was amazing! He took such good care of us and let us play our bachelorette games. Food was amazing and the vibe was on point. Thanks!,37


##### Most confident - false positives

In [12]:
most_confident_predictions(preds, "fp")

,category,text,num_words
0,fp,"Congratulations! Golden Nugget won first place in my book for WORST service EVER. The food was just ok... I think Pic-A-Lilli wings are way better... and their service shows they actually care about their customers. We showed up at 645. We ordered drinks at the bar which was good, nice impression from the bar selection and service. Then decided to grab a table around 700. We were not greeted at all... we looked right at the waitress and she didn't even notice us. At this point is a 730 and s...",163
1,fp,"When it comes to oysters I've been an absolute Apalachicola loyalist for decades. But that loyalty was severely tested tonight. At several friends' urging, we walked past famed Acme and the herd in line and strolled straight into Felix's Restaurant & Oyster Bar across the street for possibly the best bivalves we have ever slurped. Huge, salty, no crackers offered or needed. Friendly shucker kept them coming - and seemed to lose count as he exceeded the ordered dozen by at least another. But ...",258
2,fp,"My opinion of Taco Dirty has changed over time. Initially, the experience was good. The food was good, several options, a little pricy but worth it. I always got the All In bowl, and I left satisfied. \nFast forward: my last five visits have gotten progressively declining in value. To be more specific, I am paying the same and the food volume has reduced by about 25%. I am watching these young servers fumble with measuring spoons in an attempt to put strict limitations on what goes into a bo...",209
3,fp,I don't take giving five stars lightly so I will be very clear on the reasons I have chosen to do so. This restaurant came to my attention by my cousin and fellow foodie (CF). CF and I enjoy delicious food from a large variety of restaurants around the city and surrounding area. On an unspecific day last week CF came by with a leftover Spider Bao Bun from Ban Ban. Hours later the the Bun remained full of flavor and delicious. If you are a fan of soft Shell Crab they honor it at this res...,242
4,fp,"Super tasty batter, salty & spicy, but quite greasy. \n\nAs we approached, gas station paper-bagged beers in hand, we were passed by a young dude in a tank top, munching his Willie Mae's and exclaiming, ""How y'all doin'? You gonna love that chicken!"" He was speaking the truth.\n\nThere was a lineup of tourists and a sweet chromed out fixed gear bicycle with a shark on the handlebar. The line went quickly and we were seated in the back.\n\nService was quick and friendly, and the room tolerate...",344
5,fp,"I have had this place bookmarked for a while, so I was excited to see what all of the hype was about! I arrived around 11:30 on a Friday and there was already a line out the door...Great sign! I stood in line for 5-10 minutes, it moved pretty fast. I ordered the #5 - Cheesy-Veggie Po-boy. For some reason, I assumed it was served hot and I was wrong. I waited to open the wrapper until I got home. It was HUGE! The bread was so fresh and tasted a little like Jimmy Johns, but better. The...",194
6,fp,"This could easily be 4 stars if Pat's was... cleaner. \n\nI stopped by Philly during my tour de East, and of course I had to try both Pat's and Geno's. I think I tried Geno's several years ago, but I can't be sure. Anyway, after stuffing my face with a mediocre sub from Geno's, I headed across the street to meet my friends at Pat's. I ordered a cheesesteak with onions and wiz. The cute cashier took my cash, handed me change and then with NO gloves (or soap), grabbed by sub, smoother it with ...",211
7,fp,"So this place is part of New Orleans history, it is literally an institution, i don't think there is a person on staff who has been working there less than a decade and most of them are sired by past waiters, they have family legacies there and the service reflects it. Antoine'se has something like 10 dining rooms and they are all always full so make a reservation. This place is an experience and a lot

##### Most confident - false negatives

In [13]:
most_confident_predictions(preds, "fn")

,category,text,num_words
0,fn,Best hidden secret for sushi in town\nA Riesling paired\nwith some great sushi I had the volcano roll which was absolutely delisch and so is the Bubba Gump and Mexicali Blues and pretty much everything else! Deserts.... To die for yummy .....,43
1,fn,"I really do like this place, but you need to note the hours are not what they state. The hours are from 9am until whenever they sell out. This could be noon-5pm. Therefore, I would recommend going early or at least calling/reserving an order before you go.",47
2,fn,They give very good service and make good coffee comparable to starbucks.Starbucks. They also have rewards. My husband really likes it.,21
3,fn,I gave 5 stars because I either like something or I don't. \nNormally I'd give El Rey 3 Stars because it's just not my type of place. A tequila bar.,30
4,fn,"Nice pub & definitely a Pub as opposed to a bar. Went in December & they had LIVE Christmas music playing quietly in the background. Had Fried Clam Strips - they were good & the serving was huge - almost entree size. Had haddock & veggies - both were very tasty & while the Haddock was a generous portion the veggies were relatively small but that is the ONLY issue I had with the food. The service was polite, prompt & cheerful. I will return.",85
5,fn,Saw this on yelp. The staff was very friendly and the Burnt Ends Brisket sandwich was awesome. They have BBQ sauce at the table if you want to add some.,30
6,fn,Shake Shack always hits the spot. Although it's a little more pricy then say Five Guys the burgers are really good quality meat and their crinkle fries are sooooo good! The cherry blossom shake is AMAZING try it before it's gone!!,41
7,fn,Great Food. Nice Decoration. Ramen is must have. For sushi appetizer the monkey brain and the roll name Market st. Roll are sooooooo Good. Will be back.,27
8,fn,"Well, I love, love, love ABC Pizza!! The food is great! The rolls are to die for, the Greek salad with house dressing is wonderful, the pizza is good and the spaghetti is wonderful. The interior needs some updating, but I can get past all of that for the food. Highly recommend!!",52
9,fn,"I recommend to sit outside, unless is too hot. The services was excellent and the ""Heart attack burger"" is to die for. The Tri Pit and the ribeye are good as well...but my favorite was the sweet potato fries. Ohh and the view at night is nice!!!!!",47


### Amazon small model

Trained on the Amazon small dataset of ~50k book records.

#### Same-domain test set

In [14]:
model_name = "amazon_finetune"
results_fp = results_dirs[model_name]

preds_path = os.path.join(results_fp, "amazon_test_predictions.csv")
preds = pd.read_csv(preds_path, encoding="utf-8")
test_set = datasets["amazon_small"]["test"][["reviewText", "label"]]
preds["text"] = test_set["reviewText"]
preds["label"] = test_set["label"]
preds["num_words"] = [len(t.split()) for t in preds["text"]]
preds.head()

,pred_prob_0,pred_prob_1,yhat,y,category,text,label,num_words
0,0.407339,-0.571306,0,1,fn,"""At Wick's End"" is Tim Myers' first book in his candlemaking series and I had ordered it expecting a light, fluffy read. I was not familiar with his work, but thought that I might learn a little about candles and enjoy a pleasant mystery as well.I thoroughly and completely enjoyed this book! And while, yes, I did learn a lot about candlemaking (at least enough to want to make me want to try the craft), I was absolutely enchanted with the characters that Tim Myers has brought to life!This i...",1,256
1,-1.550216,1.757635,1,1,tp,"This is a great book that help you understand how to reprogram your mind in every way. You want to success in life, love, career, wealth and everything. This book could guide you to success in everything that you want. But the most important thing is you have to believed in the power of affirmations. If you ask me, yes I believe. Well, Good Luck! :)",1,66
2,-2.520277,2.908564,1,1,tp,"After reading this book, I don't know what was stranger, a famous person's shadow double-life or society's hostile attitude on mixed marriage. Either way, the title `Passing Strange', is appropriate; it means `SURpassing strange', an old phrase enjoying a comeback these days. In a nutshell, an internationally known and respected white scientist, Clarence King, a close personal friend of the US Secretary of State, posed as a black porter so he could marry the girl of his dreams, a black lady ...",1,373
3,1.835816,-2.379064,0,0,tn,This story picks up several years from where The Choice left off. The two main characters in The Choice Sol and Mattie are in this book as well. Very good storyline and each person learns something from the &#34;English&#34; as well as the &#34;Amish&#34;. Delightful reading!!!! I could not put this book down after I started reading it.,0,58
4,-2.354281,2.498593,1,1,tp,"As a father of a two 20-something kids myself, I was completely taken by his frank, earnest account of his growing up in what many would consider the ideal American family, unremarkable in many respects except for two things: 1. His parents are two women, and 2. They have an uncommonly wise approach to child rearing and life. Many straight parents would do well to read this book just for parenting tips alone. The fact that a young man (he was 20 when the book was published)can see things so ...",1,112


##### Accuracy

In [15]:
accuracy(preds)

0.7555

##### Classification report

In [16]:
with open(os.path.join(results_fp, "amazon_test_classification_report.txt")) as report_file:
    print(report_file.read())

              precision    recall  f1-score   support

           0       0.76      0.74      0.75      2988
           1       0.75      0.77      0.76      3012

    accuracy                           0.76      6000
   macro avg       0.76      0.76      0.76      6000
weighted avg       0.76      0.76      0.76      6000




##### Confusion matrix - raw & normalized

In [17]:
confusion_matrix(preds, False)

,Pred = False,Pred = True
,,
Actual = False,2214,774
Actual = True,693,2319


In [18]:
confusion_matrix(preds, True)

,Pred = False,Pred = True
,,
Actual = False,0.3690,0.1290
Actual = True,0.1155,0.3865


##### Word count comparison

In [19]:
preds.groupby(["category"])["num_words"].mean()

category
fn    175.418470
fp    167.114987
tn     78.842818
tp    270.385511
Name: num_words, dtype: float64

##### Most confident - true positives

In [20]:
most_confident_predictions(preds, "tp")

,category,text,num_words
0,tp,"Alice Hoffman's PRACTICAL MAGIC is easily her most popular title, yet she hasn't written anything similar to that book in many years, much to the dismay of her many fans. Subsequent books have ranged from disturbing to unfulfilling, as she tackled such powerful themes as spousal abuse, hazing and identity change.But this book returns her to the genre that suits her best: magical realism. The wonderful book spins the tale of a family of magical women, who pass psychic gifts from generation to...",167
1,tp,"The best single volume history of the Civil War ever written, James McPherson's BATTLE CRY OF FREEDOM was an instant critical and popular classic and a Pulitzer Prize winner upon its publication in 1998.It seems impossible, but McPherson manages to shoehorn virtually every element of Civil War America between the covers of this book. That he does so evenhandedly and without losing the narrative thread for even a moment is amazing.McPherson is a natural-born writer and historian whose ability...",292
2,tp,"From any other author, the craft and ease of these five stories would merit four stars at least, but Ishiguro has set his own standards so high with books likeNEVER LET ME GOthat he may disappoint readers with this slim collection. Subtitled ""Five stories of music and nightfall,"" the tales do have an impressive unity of theme. The protagonists are all musicians, generally putting higher ambitions on hold to play in cafe orchestras or pick-up groups; like the butler inTHE REMAINS OF THE DAYth...",464
3,tp,"This book attempts to set the record.It is amusing to find a poor argument repeated to such an extent, as though a logical fallacy, if stated over and over again, will somehow ""evolve"" into a reasoned argument. The behavior of a given individual or group holds no necessary implications regarding the validity of their principles. For every imaginable belief system you can find people who act consistently with their principles or not.There's nothing new here and you would have to have a weak f...",179
4,tp,"Although Mr. Morris goes into minute and somtimes mundane details of TR's life, I was never bored. Mr. Morris had, of course one on the most extrodinary characters in our history to work with. TR amazed his contemporaries &amp; still captivates us a century later. He is a giant &amp; it's very obvious how far we &amp; our leaders today have fallen from his standards. Morris does not cheerlead for TR. But recounting his early life points the direction that even the people of his own time...",125
5,tp,"If you play poker for money, you need to get this book. A clearer guide to poker success does not exist. In my opinion, it is better than Harrington's Volume II which is its closest rival.What makes it better are the extremely detailed post-flop hand analysis. Post-flop is where most of the money is won or lost, it is the hardest part to learn, and, not lending itself to a hand and position ""re-raise, raise, call, fold"" chart, it is the most difficult to teach.Rather than just codify a lis...",164
6,tp,"This is not just a bad book. It is a sick one.Bloom's obsessions are clear on almost every page: sex and rock music. Although clothed in a pretentious philosophical language, his objections betray what's really on his mind: he feels left out. The sexual revolution of the Sixties passed him by, and like the child whose playmates decide he's not good enough to get into the game, he retaliates by labelling everything about his opponents evil.The poet Philip Larkin had Bloom's number when he...",299
7,tp,"This was a fascinating book. It was truly fascinating and yet so depressing. It's quiet and moody and well, romance noir. The aliented, doomed hero and the femme fatale. Corruption, madness, and moral breakdown. It's all here. And it's sometimes hard to read.Claiming The Courtesan starts with the incomparable beauty Soraya and her protector the Duke of Kylemore, Justin Kinmurrie enjoying a post

##### Most confident - true negatives

In [21]:
most_confident_predictions(preds, "tn")

,category,text,num_words
0,tn,Gave to daughter-in-law as a gift. She likes to read Nicolas Sparks literature. Her daughter will read also.A gift for both.,21
1,tn,I am collecting Ellen Burns books and when I find one I like I add it to my quilt book collection.,21
2,tn,this si one of the best goosebump books that r.l. stein wrote. it's my favorite,15
3,tn,"This one actually also teaches telling time, something I did not know when I bought it. My son enjoyed this one as well. Great idea",25
4,tn,This is the book that hooked me in the Dvans family books. Enjoying them all as they are good reads.,20
5,tn,"Lots of recovery, some good humor, and super relatable despite not having kids of my own. Worth a read. Period",20
6,tn,This is a fun summer read. Light and humorous. I always need a good beach book and Carol Snow books fit this bill every time.,25
7,tn,How huge the night was agood story line. Any age could read this story and could learn a valueable lesson in life,22
8,tn,love Susan Wiggs and was glad to find that she was writing another series. love the setting and the characters in the book.,23
9,tn,A good read did not put it down I enjoy true stories better than fiction. i especially like to read a book before seeing a movie,26


##### Most confident - false positives

In [22]:
most_confident_predictions(preds, "fp")

,category,text,num_words
0,fp,"I wish I could give this book 5 stars, because the main thesis it presents (global cooling for at least the next few decades) looks right, and the main response (thorium nuclear power) looks right. That said, Archibald loses credibility with his political views, especially that the global warming myth is promulgated by the UN in attempt to take over, or at least control, world food production. That is one of the more inane conspiracy theories around. It would require a degree of UN unanimity...",222
1,fp,"I bought D.A. Carson's popular book titled ""Showing the Spirit (A Theological Exposition of 1 Corinthians 12-14)"" several years ago and waited with great anticipation for the opportune time to finally read this book on one of the most controversial subjects in the church today. My first observation about this study was the unusual book title. Carson doesn't let the reader in on this point until nearly the end of the book. This book was published twenty-five years ago in 1987, back when some ...",1567
2,fp,"OR WANNA-BE QUILTERS!Reversible Quilts by Sharon Pederson is absolutely filled with patterns with simple, easy to follow directions that produce the greatest results! Sharon Pederson has removed the mystic and shown how easy it is to have stunning reversible results. I recommend this book for anyone who quilts or would like to.",52
3,fp,"1984 by George Orwell is a great book, but Brave New World by Huxley has captured the sad spirit of todays world. Some elements such as flying cars are either non existent or in infancy stages, but the central ideas are alive and well in America. With a world that appears to be growing more violent by the day, more Americans are looking for a place to escape--- a utopia. With the media saturation with images of sex, it's almost like children are indocternated with the idea of sexual play fro...",264
4,fp,"Laurie Notaro has outdone herself this time. Always interesting and always hilarious, she has raised her dryly offbeat humorous look at life to a new level with this book. Telling a good short story is a fading part of the American literary experience, and she, and the others that still practice the art of big ideas with few words, should be followed carefully and encouraged, lest they be lured away from their craft into the glitzy (and, no doubt more profitable) world of creating dull, as...",123
5,fp,"After listening to Pastor McConnell's testimony several months ago and seeing his autobiography for three bucks, I dropped the Washingtons for this incredible account of how the Father radically tranforms people's lives. Is There Anybody Out There? is not your typical Christian story, and there is very little reference to God in the first half of the book. Pastor McConnell had an extremely troubling upbringing that led to a life behind bars yet God got a hold of Him and changed him forever. ...",209
6,fp,"""The Autobiography of Mark Twain,"" Volume I, has been worth the wait. In spite of the 200+ pages of Introductory material, the Twain writings are excellent. Hemingway claimed he was the best American writer, and I think he was correct. I've read a great deal of literature over the years, but I don't believe there is another American writer with the range of emotions, the wit, the unbelievable insight into the human condition that Twain demonstrates over and over. His life was filled with...",143
7,fp,This is an authentic historical record of how the horses changed the lives of Natives living on the plains of America. The horses came from runaway from explorers and tribal interaction. It is interesting how the horses and the Indians both were examples of survival of the fittest. They went from subsistence hunter gatherers to more efficient yet more respectful of nature societies. From having to drive a herd over a cliff when they were on foot to selecting individuals as their prey made f...,187
8,fp,"I spent 4 years studying Latin in high school in the 197

##### Most confident - false negatives

In [23]:
most_confident_predictions(preds, "fn")

,category,text,num_words
0,fn,"I will be looking forward to the next books that John Cassell writes. If u are looking for a little different read,this is it. I found it very intereating. 1969 was a strange,different time to come of age in,it expresses the times and all the changes well. Enjoyed very much. Look forward to whats to come.",56
1,fn,is world reknown for his historical books in a narrative format. This is not dry history. This is living history.,20
2,fn,Enjoyable reading. Looking forward to the next adventure in the series. Story line jumps around a bit but reveals itself in a way that makes the story more enjoyable in the end.,32
3,fn,I loved reading this book even through it was short. I had me on the edge of my seat. I was just like I was watching a horrow movie. I read this book and night after my kids went to bed and I swear I heard the same noise that was in the book lol. This is a must read book!,61
4,fn,"This is not a biography. This is how the author thinks he feels about his life, his music, his passion.",20
5,fn,I grew up in Jersey City and knew many of the players and places in the book. I remember when Frankie got shot. It was a nostalgic read for me.,30
6,fn,"I like this book. It has several patterns that I hope to make. Writing is clear, pictures are good.",19
7,fn,"This book is a great resource for parents and has been enjoyed by everyone in my family-- ages 4,8,11, and 15. My husband and I have enjoyed the stories as well. I highly recommend it.",35
8,fn,A good paranormal romance novel about a girl named Ana who falls in love with a shapeshifter named Chance. There are some minor grammar errors but it's an enjoyable read.,30
9,fn,"Totally freaking awesome! I am not really into the typical Romance novel, and this book made me want to get into them! NA is the next best thing, so you should get on the wagon now and CHECK IT OUT!!! I'm in love with this whole book :)",48


#### Different-domain test set

Using the model to predict the Yelp test set, without any additional fine-tuning.

In [24]:
preds_path = os.path.join(results_fp, "yelp_test_predictions.csv")
preds = pd.read_csv(preds_path, encoding="utf-8")
test_set = datasets["yelp"]["test"][["text", "label"]]
preds["text"] = test_set["text"]
preds["label"] = test_set["label"]
preds["num_words"] = [len(t.split()) for t in preds["text"]]
preds.head()

,pred_prob_0,pred_prob_1,yhat,y,category,text,label,num_words
0,-1.889789,2.398464,1,1,tp,"There is a small, evil part of me who really wants to give Charlie's 1 star and a terrible review just so you won't go there and find out about it's goodness. It isn't really big enough to handle the over-capacity of being popular among more folks then the few who are ""in the know"". \n\nBut then I realize that it's not fair, Charlie's is awesome and still kinda hard to find and if you make the effort to get yourself there then you deserve to experience the aforementioned goodness.\n\nCharlie...",1,270
1,0.862482,-1.543958,0,0,tn,Really great the whole experience was amazing had many people refer me to this place and it was worth it all,0,21
2,-1.498450,1.883841,1,0,fp,"One of the few, if not the only restaurant I've been to more than 10 times in Philly. Just amazing!\n\nYou can't say enough good things about the Tahini Shake. They're all great, although my favorites are the coconut and the mint chocolate chip. \nThe fries are perfectly crunchy too and the seasoning is great. \nThe staff are so incredibly nice. \nThere is a small shelf of grocery type items you can buy if you would like as well.\n\nCons:\nIt closes at 7PM. \nSometimes not enough seating. \n...",0,90
3,0.599810,-0.669705,0,0,tn,The lark is a local favorite and it's on every list of top restaurants in Santa Barbara. We really wanted to love this place and had high expectations based on what we read. \n\nThe ambiance is very warm and welcoming with a winding brick outdoor patio lined with iron fireplaces. We went on a Wednesday with reservations and we were glad we did because it was buzzing. \n\nThe catch here for us was the food. We were told that the style was small shared plates and multiple were recommended but ...,0,235
4,-0.551172,0.677422,1,1,tp,"Where to start? This place has so much potential; a great atmosphere, good drink selection and decent food. Where they lack is in execution. \n\nOur most recent experience:\nOrdering a drink should not take 20+ minutes, especially when those drinks are a beer & a glass of wine. Requesting ketchup should not be a 15 minute process after your food comes. Having your server pulled to cover the bar happens, but for no one to pick up their tables is awful customer service; attempting to make up f...",1,148


##### Accuracy

In [25]:
accuracy(preds)

0.6004411265693926

##### Classification report

In [26]:
with open(os.path.join(results_fp, "yelp_test_classification_report.txt")) as report_file:
    print(report_file.read())

              precision    recall  f1-score   support

           0       0.65      0.47      0.54      2992
           1       0.57      0.74      0.64      2902

    accuracy                           0.60      5894
   macro avg       0.61      0.60      0.59      5894
weighted avg       0.61      0.60      0.59      5894




##### Confusion matrix - raw & normalized

In [27]:
confusion_matrix(preds, False)

,Pred = False,Pred = True
,,
Actual = False,1405,1587
Actual = True,768,2134


In [28]:
confusion_matrix(preds, True)

,Pred = False,Pred = True
,,
Actual = False,0.238378,0.269257
Actual = True,0.130302,0.362063


##### Word count comparison

In [29]:
preds.groupby(["category"])["num_words"].mean()

category
fn    159.641927
fp     99.524890
tn     57.226335
tp    213.822399
Name: num_words, dtype: float64

##### Most confident - true positives

In [30]:
most_confident_predictions(preds, "tp")

,category,text,num_words
0,tp,DO NOT EAT THIS BREAD IF YOU ARE A CELIAC!! My daughter and I ate this bread and were sick for weeks. I called the company and they claim to bake gf bread once a week and clean throughly after baking gluted filled bread. THEY DO NOT DO A VERY GOOD JOB CLEANING THEIR FACILITY OR THEY ADD SOMETHING NOT GF TO THE BREAD. I did some research and found several celiac support groups who contacted this bakery and asked them to either stop advertising their bread as gluten free or clean their facilit...,131
1,tp,"Should Reno stop it with the AYCE sushi bars? I don't think so. More competition, the better. What Hinoki does here is opposite of a lot of sushi bars in town. You will not find a menu with a hundred rolls, at which point you can't remember what's good and what's not. You won't find a list of ""specials"" of exotic fish that are just that, exotic. No one ever said exotic had to actually taste good and while having rare type of sushi is fun, none the less, I don't come here to play. I came for ...",245
2,tp,"I really am unsure as how to proceed. How to relate to others the absolute disdain I have for this restaurant and especially the wife of the Owner. This woman is basically the ""soup nazi"" in pumps...\nI have made reservations for every foray into this place and have tried to give them every benefit of the doubt. However, even though we had reservations for a particular time one evening, it was never written down and I even explained to her to whom I had spoken. It didn't matter; it was the m...",370
3,tp,"If you want disaffected trustafarians or hipsters to serve you overpriced coffee at their own leisurely pace, this is the place for you.\n\nComing off of living in Portland, Oregon for fifteen years, it's as if these jamokes were relocated to Philadelphia, since they couldn't hack it in the Pacific Northwest as a barista. \n\nEverything about this place needs to be changed from the pricing to the seating and most importantly; the staff. I strongly suggest that the owner learn the distincti...",91
4,tp,"I go to Split Rail almost weekly, they constantly have the best beer selection around, they are simply killing it. The food is also fantastic. Not sure about the other reviewer who mentioned it not being spaced out/socially distanced, as I mentioned I am there almost weekly and they definitely space out the tables and the crowd is socially distanced. Everyone is wearing masks as well.\n\nI am glad to see Split Rail making it through these trying times, it would be a huge loss if they were n...",126
5,tp,"BUYER BEWARE. Even the reviews tell the truth:\n\nAmong the 5-star reviews I found this consistent theme: 4 reviews, 3 reviews, 2 reviews, 4 reviews, 2 reviews, 1 review and 1 review. Hmmmmm... could these be friends of the owner?\n\nYet take a look at the seasoned reviewers... Hmmmm... not very many 5-star reviews - I think one. Almost all are less than 5-star and several are critical.\n\nLiving in Tampa spoils some of us. We know where to find great Cuban food, and in almost every ...",193
6,tp,"Ok, I got in trouble with my friend Dave who owns Pho orchid for my dalliances into other pho joints! Sorry Dave, I just wanted to see if the grass was really greener on the other side! Besides, I usually get the vermicelli with the lemon grass bed with an added eggroll on top. Omfg is it good! The beef is seasoned just perfectly, ahhh... I'm having a moment! For appetizers I call ahead and ask them to get my chicken wings ready bc they take a whole to make but they are SO worth the wait! Th...",601
7,tp,"Just left Beachcomber about an hour ago and have to say for the first time in probably 8 years we were very disappointed. We've been coming to Clearwater on vacation for about 13 years. Had been frequenting another high end restaurant but decided to check out Beachcomber based on recommendations several years ago. We love the feel of the place and the food has always been memorable. Tonight opted fo

##### Most confident - true negatives

In [31]:
most_confident_predictions(preds, "tn")

,category,text,num_words
0,tn,Always great bbq & local makes it even better. Have Scarlet Lane attached is a plus for a great beer to go with it,24
1,tn,"Brendon the bar tender is awesome, always friendly and the food is always good! The other customers are alway friendly and this is a great place just to unwind!",29
2,tn,"San Jose Crestwood always has great food and service. They are fast, friendly and food quality is consistent. One of my favorite restaurants.",23
3,tn,"I love this place. You can feel the history lurking before you even walk in. I did not eat here but they are suppose to have the best mufaletta in town. It's on the other side of town from where i live so I won't be a regular, but I will go back. I enjoy anywhere where there could possibly be a ghost siting and this is one of those places.",71
4,tn,Such great flavor and great customer service. It's worth driving to the weird location.,14
5,tn,The best bbq join in Nashville! I went back 3 times in a day ! Definitely will go again next time visit Nashville,23
6,tn,3 stars but due to the company I was with it is more like 4. I normally would not find myself at an establishment such as this but we had some drinks and had a good time. (This was in June) I cannot even IMAGINE the madness that probably goes down on game day.,54
7,tn,"Great neighbourhood restaurant with excellent tasting dishes. \nThe staff were quite friendly and attentive. \nPrices were average. \nAtmosphere is quaint, sort of loud, but like at home with the family.",30
8,tn,Always good fun here I love the Djs playlist at night hip-hop r&b and classics #90skid,16
9,tn,"One of my favorite places to stop when I have a layover in Philly. Food, service & drinks are always great!",21


##### Most confident - false positives

In [32]:
most_confident_predictions(preds, "fp")

,category,text,num_words
0,fp,"Like any Buffett at this price point, don't expect the world! And like all buffet's there is a better way to do it than not. Oh, don't think disliking this location justifies disparaging comments. $7.95 for lunch! When you can, get there when it opens, it's unlimited visits, ie.. put less food on your first plate, be selective, keep an eye open for items being replenished or replaced, sit closer to the buffet when you can, wash hands before touching the utensils. It's far from perfect, but k...",96
1,fp,"To the credit of the restaurant - we went on a Sunday afternoon when it was crowded and their power had gone out earlier (?). However... the employee making the sandwiches was moving at a snails pace and showed pretty much no effort to make the sandwiches with care. After the first 30 minutes of waiting, I came inside to see what was going on and proceeded to wait another 20 minutes while I watched him slap together sloppy sandwiches with Mayo and oil all over the outside and ingredients fal...",149
2,fp,Thank you KFC for not posting a sign that the dining room closes an hour before the drive thru. Thank you KFC for not saying anything until your ready to take my order. Thank you KFC in Pinellas Park for suggesting I go get in the drive-thru line that wasn't moving. Thank you KFC customers that were still in the dining room that you had already been waiting 30 minutes for your order. And thank you KFC for making it easier to choose someplace else where they can serve all their customers.,92
3,fp,"This restaurant lacks fun and life! Food was terrible for the price! Ordered chicken tacos for 13.95 and got two small tacos and the corn tortillas were flopping around like they had been cooked earlier and reheated in the microwave! Got a side of salsa that' looked like days old pico. Everything we ordered tasted bland and not authentic fresh Mexican food! The bathrooms were dirty and disgusting! The floor mat was covered in dirt. Service was not that good either, waitress acted like she co...",111
4,fp,The worst food experience. Apparently they call peaces of fat strips as smoked brisket. Coleslaw salad looks like a coleslaw soup. If you want good smoked meats go elsewhere.,29
5,fp,"I like the hipster friendly vibe. We've been here 2 days in a row and here are the results: \n\nMocha Frape 10/10 - I strongly recommend this if you're a frape enthusiast. Super sweet though. Portion size for the price was not enough for me, but it was so good.\n\nThe Harvest 10/10 - Looks cool, tastes even better. I didn't know it came with ice cream because the picture on the menu didn't show, but then again I didn't read, I just pointed at the picture.\n\nMatcha Coconut Latte 7/10 - Ehh ...",275
6,fp,"This Thai is really awful. There's no flavor and my favorite dishes that I've had a twenty different restaurants are basically just barely flavored onion dishes. Plus they keep making my dishes wrong and making us go back to fix it. They don't seem to really care about making the customer happy either. Overall a very disappointing experience , over and over again. Their sushi is okay, but does not make up for the miserable excuse for Thai.",78
7,fp,"Literally the WORST Mexican food I have ever had. I was highly recommended this restaurant by a friend so I went.\nDry rice, cold beans, tasteless veggies, water on the floor! Bathroom has tape on the floor drain? ARE YOU SERIOUS?",41
8,fp,"I stopped in with a pal for a cuppa java yesterday. I specifically asked for Decaf as I cannot have caffine.\n\nI should have known when I went in the joint it was pathetic as there were no customers inside it to speak of. \n\nThe server brought me caffinated coffee. Lucky for me I only had one cup. I can't discern caffinated from decaf generally. Had a cinnamon roll maybe baked a week or two ago, was ok if you like dried out used up pastry.\n\nThe server asked to be paid before we wer...",186
9,fp,"The food is overpriced and bland. I agree t

##### Most confident - false negatives

In [33]:
most_confident_predictions(preds, "fn")

,category,text,num_words
0,fn,This is literally the bar where everyone knows your name. Great local crowd. It can get busy some Saturdays and on 3rd Friday. The staff is awesome on Friday nights but I'm probably biased. :),35
1,fn,Been going here ever since I moved into the area. Lee's Deli is a classic and great to grab a variety of food from. I recommend it to everyone. The owner quality controls everything at the front and I've never had any issues with orders.,45
2,fn,"Pricey, but that's expected. Very small interior with some outdoor seating. Refreshing, especially on a muggy day walking around Music City.",21
3,fn,"Always high expectations when coming here, and they always deliver. Top-notch food flavors drinks and service can't be beat.",19
4,fn,I am so amazed at what this bakery can make with their cakes! My mom has bought birthday cakes from here and I have also bought a cake here for my girlfriends birthday. I sent them a picture of hello kitty and they made the exact cake I asked for red velvet and butter cream and it came out just as amazing as it sounded! I will always use this bakery for their epic cakes for my family and special birthdays,81
5,fn,Always come here whenever we go to New Hope - candle shop is great as well as Coach - always have to pick up something cute as you honestly cannot beat the prices (few weeks ago i got a $400 bag for $160) hit up the little cafe for lunch and a dairy queen cone for dessert - perfect!,59
6,fn,This is a great brewery a little off the path which is what makes it kind of cool. Great seating outside and the beer is very good. Highly recommended.,29
7,fn,We've gone a few times. Service is always lacking. Food is never great.,13
8,fn,Had dinner on a Monday night with the hubby. Always night when a restaurant was open on a Monday. My hubby wasn't too pleased with the service which was lacking a bit but nothing too scathing. The food was pretty tasty but did give us a bit of a sour tummy later... think I ate too much of the yummy mozzarella appetizer hehe. The patio area is the best part. Very pretty.,72
9,fn,"Went here for brunch and it was fantastic. The greeters where all smiling and very welcoming. Love the interior design as well, felt very homie southern vibe with a twist. Caty F our waiter was phenomenal. Was very patient explained the menu to me and recommended the fried chicken biscuit sandwich, which was delicious. I would definitely come back to this place.",62


### Amazon large model

Trained on the Amazon large dataset of ~250k multiple-domain records.

#### Same-domain test set

In [34]:
model_name = "amazon_finetune_LARGE"
results_fp = results_dirs[model_name]

preds_path = os.path.join(results_fp, "amazon_test_predictions.csv")
preds = pd.read_csv(preds_path, encoding="utf-8")
test_set = datasets["amazon_large"]["test"][["reviewText", "label"]]
preds["text"] = test_set["reviewText"]
preds["label"] = test_set["label"]
preds["num_words"] = [len(t.split()) for t in preds["text"]]
preds.tail()

,pred_prob_0,pred_prob_1,yhat,y,category,text,label,num_words
24995,-1.094314,1.423536,1,0,fp,I usually LOVE edge of your seat novels.... but this one was just WAY TOO LONG! 117 CHAPTERS????? Are you kidding me? Would hate to get into a conversation with this author!,0,32
24996,1.285419,-1.914878,0,0,tn,"This product installed Okay but the software is not completely compatible with Windows XP, as a result on occasion it crashes my system on boot up. In order to use, you'll have start your system first, then connect it to your PC - wait for software fix or upgrade.",0,49
24997,-0.825702,1.197638,1,0,fp,"I became a fan when I got Power & Passion on cassette back in 1985. Great album! It's hard to find it on cd so this collection will have to do for now. This collection includes 6 tracks from P&P as well as great tracks from other albums like Runaway Dreams, MWACN (Slade), and Last Thing At Night (instrumental). I wish it had Running Away from Growing Up The Hard Way though. Pat McManus is a very good guitarist. His solos are always very melodic and well structured. I hope Power and Passion i...",0,96
24998,-0.547870,0.781104,1,0,fp,"I'll keep this simple, this is one of the best sounding DVD concerts I own. The DVD package is good with extra footage and interviews. There is CD included in the set and the concert is excellent. All the right songs, played by a good line of people. Luke is on with all of his guitar work and the band is right there on top of things.",0,67
24999,-1.410084,2.074712,1,1,tp,"This 2-disc set, ""The Complete A&M; Recordings"", is actually four albums Minnelli recorded for the A&M; label between the brief period of 1968-1970 , along with alternate recordings & unreleased cuts.The first two albums, ""Liza Minnellli"" and ""Come Saturday Morning"" are quite lovely and find Liza in a mellow, pop-folk early-70's mood, handling this material very well, considering that her forte has always been, and will always be that of the bombastic, socko-boffo-POW Broadway style of singi...",1,400


##### Accuracy

In [35]:
accuracy(preds)

0.74332

##### Classification report

In [36]:
with open(os.path.join(results_fp, "amazon_test_classification_report.txt")) as report_file:
    print(report_file.read())

              precision    recall  f1-score   support

           0       0.79      0.66      0.72     12453
           1       0.71      0.83      0.76     12547

    accuracy                           0.74     25000
   macro avg       0.75      0.74      0.74     25000
weighted avg       0.75      0.74      0.74     25000




##### Confusion matrix - raw & normalized

In [37]:
confusion_matrix(preds, False)

,Pred = False,Pred = True
,,
Actual = False,8216,4237
Actual = True,2180,10367


In [38]:
confusion_matrix(preds, True)

,Pred = False,Pred = True
,,
Actual = False,0.32864,0.16948
Actual = True,0.08720,0.41468


##### Word count comparison

In [39]:
preds.groupby(["category"])["num_words"].mean()

category
fn    122.620642
fp    164.945008
tn     57.824367
tp    278.452686
Name: num_words, dtype: float64

##### Most confident - true positives

In [40]:
most_confident_predictions(preds, "tp")

,category,text,num_words
0,tp,"I was on the fence aboout this, not becuae it's not an Ipad (ipad doesn't qualify to enter my house yet), but because of some negative reviews on wifi and touch screen.Frankly, I have not had any of these issues. I loved this tablet from the moment I opened the box.The awesome:Honeycomb is truly an OS that was concieved for a tablet PC, and it shows. Everything works, I am typing this review on the tablet and love the keyboard. Love the full browser Love the fact that you can plug in a USB s...",224
1,tp,"Let me qualify this review by making it clear that I am not a Mac user. My only experience with Mac's is a few mins spent each time I visit Apple store. I am writing this for those thinking of a switch from XP to Mac. I know it will be unpopular with Mac fans but hope it helps you XP users make the right decision.iBook pro's,1. It looks sleek, smooth and sexy. Nothing in PC world is this well designed.2. No weekly patches coming from Redmond to apply.3. No anti-virus or anti-spyware needed (...",377
2,tp,"I wish I'D have had access to something like this DVD way back when I first learned the Yang Style 24 Posture Form, but buying a VHS tape of something like this would have been prohibitively expensive, even if it had existed back then. (BLANK VHS tapes were selling for $25.00 each in those days!) The information on this DVD is well produced, easy to access, very informative, and reasonably priced. I won't go into the details of all that is included on this DVD, as all that has been well d...",284
3,tp,"Shaw's PYGMALION. like Julius Caesar's Gaul, is divided into three parts.1. A preface, which was written after the play was already a hit, but was meant by Shaw to be a part of the reader's experience, and is necessary to the understanding of Shaw's main theme.2. A five act play, meant to be performed, and which is annotated in such a manner so as to facilitate deletion, on the stage, of portions only possible in a film version.3. What Shaw refers to as a sequel, written in prose, and out...",674
4,tp,"For legendary tenor Pl&aacute;cido Domingo, this summit among Verdi's dramatic baritone roles has gone from a ""maybe someday"" dream to a daring experiment slated for the eve of retirement, to, finally, just another role for the (voluminous) books as his career chugs on in its sixth decade. In the same month Domingo celebrates his 70th birthday, two new DVDs document the undertaking, with Sony licensing the February 2010 Met broadcast and EMI the July 2010 Covent Garden. The dual releases are...",1479
5,tp,"I have to admit, that I had my reservations about this program. Based on accepted scientific dogma, the Atkins Diet seemed very dangerous.Then I bumped into a friend that I hadn't seen in awhile. He had lost around 60 lbs and looked fantastic. Then his wife came out and she had lost 45 lbs. I asked what they were doing and they said &quot;Atkins.&quot;What impressed me is that these eople didn't look like normal dieters. They looked wonderful. So radiant. So much energy. Their skin had a glo...",439
6,tp,"I am probably one of the very few people out there that did not like &quot;The Mirror Conspiracy&quot;, which in my opinion sounded very commercial, mundane and uninspired. It seemed like Thievery was stuck in a rut, overusing the sitar and other Indian music on almost every track. &quot;Mirror&quot; was too busy trying to impress every listener on every track, although the production value was still high. &quot;The Richest Man In Babylon&quot; is a comeback of sorts with a greater variet...",221
7,tp,"After owning 3 Palms and one Pocket PC (... I kept dropping them), I bought but deeply regret buying a Clie. If you are a heavy user of &quot;tasks&quot; as I am, Clie's performance is so bad the PDA is unusable. When one makes any change in the task (...change date to &quot;tomorrow&quot;), there is about a 5 second delay. For all the Palms this was instantaneous. Change &quot;priority&quot;...

##### Most confident - true negatives

In [41]:
most_confident_predictions(preds, "tn")

,category,text,num_words
0,tn,The guitar range of Joe Bonamassa shows through. Rock; Melodies; Hard Blues; Sorrowful Blues. He speaks with his music. Worth as good listen.,23
1,tn,I love Jeff Beck's music. This CD was new and the priced reasonable. It arrived quickly. I will do business with the vendor again.,24
2,tn,my sister and i both are health nuts with a love for juicing. i will certainly be sharing some of these ideas with her.,24
3,tn,I bought this to replace my 880 which had been dropped one to many times. It works quite well and is easy to program. My program is not 100% yet but have only done the initial setup and have been taking note of the problems in it. I believe it may need a few pauses is all.,57
4,tn,Ilove this series and would recommend for everyone. Well done and a great deal for entertainment. Who doen't love ANDY GRIFFITH?,21
5,tn,good to have,3
6,tn,"Jeff Dunham's DVD's are great, I have 6 so far, they keep you laughing. It only took a few days to get to me",24
7,tn,I never used this. So I gave it to my grandson. he uses it a lot. He says it works just fine.,22
8,tn,Another RUSH album for my sister because I wore out the cassette she had.,14
9,tn,This was my husbands favorite CD so the fact that the CD was in perfect shape for such a perfect price was well~perfect!!,23


##### Most confident - false positives

In [42]:
most_confident_predictions(preds, "fp")

,category,text,num_words
0,fp,"The title says it all. This machine does everything I had hoped for with a very intuitive user interface and so far has worked perfectly (been using it about 3 months). What more can one ask for? One feature of this particular model is the ability to add a Secure Memory card in a recepticle located on the top of the iPAC. You don't need an extra ""slip on"" accessory to add memory with this model. I listen to mp3s that I download a great deal of the time so the extra memory (I use 256MB c...",321
1,fp,"Kale, quinoa, fibre, oils, fish, proper sleep, etc etc. You've probably heard these things for a very long time, and obviously because we still live on the same planet which has the same kind of food and stuff and their benefits do not change every year.You'll find these tips and many more just by searching online. There is no SECRET magic ingredient or technique in here.",66
2,fp,"I was put off at the time by all the hoo-hah that surrounded the publication of ""The Girl with a Dragon Tattoo"". Members of my family knowing my love for good crime writing, urged me to read it but perversely I declined. I have at last got around to it and found just what they told me I would - a book you cannot put down. Stieg Larsson had real talent to have crafted such a compelling tale from what at the start seems rather unlikely material - an ice cold case and an old man's refusal to ...",168
3,fp,"David Fincher's suspenseful and gritty thriller grips you, leaving you speechless and pondering once the end credits have finished rolling, even while watching the supplements. With its dark and morbid atmosphere and the uncompromising down-beat ending it will leave you thinking about what just sped by you. Harsh and real, &quot;Se7en&quot; is a homicide thriller that will keep you on the edge of your seat for its entire running length, and it will not fail to impress its disturbing mood upo...",355
4,fp,"Teenage Mutant Ninja Turtles Volume 4 has some great stuff on it. Baxter Stockman comes back (along with the Rat King), Casey Jones is introduced (sounding like a breathy Clint Eastwood), Shredder thinks that he's Michelangelo when a duplicating experiment goes wrong, an alien tourist almost almost sucks up all of New York with his camera, Raphael, Michelangelo, and Leonardo are given a love potion and fall in love with Irma, while Rocksteady falls in love with April, and Shredder and Krang ...",226
5,fp,"Upgraded from my old Nuvi and this is a huge improvement. The display is really clear but there can be an issue with fingerprints and screen glare. I clean it regularly and adjust the position of the screen to avoid the sun's reflection. Not a big deal for me. Voice activation works ok, I speak clearly and slowly and it seems to pick up most of the commands without a problem. The only glitch I found was adding a POI with voice command to my current route. It didn't pick up the fact that I wa...",282
6,fp,"Remix albums of world music can easily slip off the mark, offending both world music purists and techno-beat lovers. Admitedly, it takes an open mind to hear the Queen of Morna, Cape Verde's barefoot diva, remixed and pro-tooled by today's hot remix stars... if you've got an open mind, and open ears, venture in. Downtempo remix stars Chateau Flight, 4 Hero and others reinterpret Cesaria's hits with respect for the originals. If you like Madredeus' remix album ""Electronico"", or Thievery Corpo...",100
7,fp,"Tony Bennett at Carnegie Hall is a must have for.......anyone who appreciates artistry in it's purist form. Tony nails it over and over again with his exquisite interpretive styling, range, selection and emotional depth. It's pure lightening in bottle. No need to choose highlights for it's really best to put this on, lie back, close your eyes and imagine yourself to be one of the lucky souls who happen to have been in attendance on this magical night watching and listening this quintessen...",104
8,fp,"Okay, so Se

##### Most confident - false negatives

In [43]:
most_confident_predictions(preds, "fn")

,category,text,num_words
0,fn,Kris Kristofferson hits home with this CD ! It brought back thoughts of times gone by and the time to come !,22
1,fn,I bought this for an ubuntu build i made and so far have no issues with it. This worked out of the box no configuration changes needed.,27
2,fn,This is one of the best products I bought. It is a bit bulky though.Anker's vendor has always been very professional. They will answer all you questions and also work with you on the product warranty issue (other products I bought). The process is very prompt and courteous.Highly recommended for this product and Anker's products.,55
3,fn,made you roost for the actors/actoress as when fight mother nature.. good movie that kept you in suspense. will watch many times,22
4,fn,"LOVE THE CHARLES LlOYD SUPER GROUP.THE WHOLE CD IS STRONG, NO WEAK TUNES IN THE CD. A BEST BUY FOR THOSE WHO LOVE THE SOUND OF CHARLES MUSIC.",28
5,fn,"I have bought several things in the past and purchased Square Trade to cover them.The cost is very low, and I was unfortunate in a printer that I had covered died.But square trade sent me a replacement VERY FAST...No hassles no quibbles, just great service.",45
6,fn,Andy Griffith rules in this spectacular little known drama thriller.It's a must see at a great price!!,17
7,fn,Looks like any other Flash Shoe Holder. Can't tell it from any other.Broke on first use while tightening to stand.Try another from someone else.,24
8,fn,"I have always sort of liked Dylan and now that Charlie Daniels and his band have reworked Dylan's music and put it to press. Well as Phil would say, &#34;HAPPY HAPPY HAPPY&#34; that pretty much sums it up for me !!! really great to have.",45
9,fn,"So far the Bluetooth is working well. The sound is excellent for Bluetooth!. The best I have ever heard. I have noticed that music has very briefly cut out on occasion in my 1st day of use. This could be due to the positioning of the device behind my tv stand. Will have to monitor it. Easily paired, no hassle. So far I highly recommend this product.",67


#### Different-domain test set

Using the model to predict the yelp test set, without any additional fine-tuning.

In [44]:
preds_path = os.path.join(results_fp, "yelp_test_predictions.csv")
preds = pd.read_csv(preds_path, encoding="utf-8")
test_set = datasets["yelp"]["test"][["text", "label"]]
preds["text"] = test_set["text"]
preds["label"] = test_set["label"]
preds["num_words"] = [len(t.split()) for t in preds["text"]]
preds.head()

,pred_prob_0,pred_prob_1,yhat,y,category,text,label,num_words
0,-1.776873,2.609492,1,1,tp,"There is a small, evil part of me who really wants to give Charlie's 1 star and a terrible review just so you won't go there and find out about it's goodness. It isn't really big enough to handle the over-capacity of being popular among more folks then the few who are ""in the know"". \n\nBut then I realize that it's not fair, Charlie's is awesome and still kinda hard to find and if you make the effort to get yourself there then you deserve to experience the aforementioned goodness.\n\nCharlie...",1,270
1,1.534918,-2.227438,0,0,tn,Really great the whole experience was amazing had many people refer me to this place and it was worth it all,0,21
2,-0.647792,0.995099,1,0,fp,"One of the few, if not the only restaurant I've been to more than 10 times in Philly. Just amazing!\n\nYou can't say enough good things about the Tahini Shake. They're all great, although my favorites are the coconut and the mint chocolate chip. \nThe fries are perfectly crunchy too and the seasoning is great. \nThe staff are so incredibly nice. \nThere is a small shelf of grocery type items you can buy if you would like as well.\n\nCons:\nIt closes at 7PM. \nSometimes not enough seating. \n...",0,90
3,-1.414079,2.120411,1,0,fp,The lark is a local favorite and it's on every list of top restaurants in Santa Barbara. We really wanted to love this place and had high expectations based on what we read. \n\nThe ambiance is very warm and welcoming with a winding brick outdoor patio lined with iron fireplaces. We went on a Wednesday with reservations and we were glad we did because it was buzzing. \n\nThe catch here for us was the food. We were told that the style was small shared plates and multiple were recommended but ...,0,235
4,-0.121720,0.281927,1,1,tp,"Where to start? This place has so much potential; a great atmosphere, good drink selection and decent food. Where they lack is in execution. \n\nOur most recent experience:\nOrdering a drink should not take 20+ minutes, especially when those drinks are a beer & a glass of wine. Requesting ketchup should not be a 15 minute process after your food comes. Having your server pulled to cover the bar happens, but for no one to pick up their tables is awful customer service; attempting to make up f...",1,148


##### Accuracy

In [45]:
accuracy(preds)

0.6538853070919579

##### Classification report

In [46]:
with open(os.path.join(results_fp, "yelp_test_classification_report.txt")) as report_file:
    print(report_file.read())

              precision    recall  f1-score   support

           0       0.73      0.50      0.59      2992
           1       0.61      0.81      0.70      2902

    accuracy                           0.65      5894
   macro avg       0.67      0.66      0.65      5894
weighted avg       0.67      0.65      0.65      5894




##### Confusion matrix - raw & normalized

In [47]:
confusion_matrix(preds, False)

,Pred = False,Pred = True
,,
Actual = False,1496,1496
Actual = True,544,2358


In [48]:
confusion_matrix(preds, True)

,Pred = False,Pred = True
,,
Actual = False,0.253817,0.253817
Actual = True,0.092297,0.400068


##### Word count comparison

In [49]:
preds.groupby(["category"])["num_words"].mean()

category
fn    117.253676
fp    108.635695
tn     50.688503
tp    218.454623
Name: num_words, dtype: float64

##### Most confident - true positives

In [50]:
most_confident_predictions(preds, "tp")

,category,text,num_words
0,tp,"Yes!! Is there anything more I can add to the hundreds of other reviews...? Hmm, well I would point out that they have wonderfully reasonable hot lunch specials. And for fans of authentic Japanese kitchen food, it can be found here. Honestly, we don't even come here for the sushi (gasp!) but for the yosenabe, zaru soba and katsudon lunch bowls etc. If all you know is tempura & teriyaki, maybe try something different like sukiyaki or agedofu next time. Although they do still have the bes...",90
1,tp,"So I'm playing the Gas Game while driving back north to Santa Barbara from Santa Monica during my Big Bear Weekend... you know the one where you are like, ""I'll get off at the next off-ramp and get some gas""... then you are like ""I'll pull off it 5 more miles"" and then you realize there isn't anything for a few more miles and you are shitting in your pants thinking you are going to have AAA to tow you a mile to a gas station. Or is it just me? I think my mind is just playing games with it...",291
2,tp,"PLEASE READ ENTIRELY.\nSpare yourself the heartbreak and do not book here, for anything: your birthday, your baby shower, your pity party for one, and most importantly DO NOT BOOK HERE FOR YOUR WEDDING. Starting off your wedding here will not only be disappointing, it will disastrous.\n\nI got married on Sunday, Sept 23, 2018. Aiden and I communicated regularly throughout the planning process. The restaurant offers multiple options of set menus, in which my husband and I opted, not only for ...",835
3,tp,"The saddest review I've ever wrote:\n\nAbsolutely terrible! \nFirst they told us a 35-45 minute wait... when it actually was only 10 minutes... \n\nThey ended up giving our table away only after 20 minutes. The funny part about it is that they made us feel bad about it because they told us it was longer than it really was. We went by the lake and saw the sunset while we waited. As a suggestion, I think they should start taking numbers so that people know when their tables are ready.\n\nFrom ...",298
4,tp,"Don't believe the hype!!\nHighly Over rated. Should have stuck with my 1st mind. Can't go wrong at Stein's.\nThe fried bologna sandwich is a joke. 2 pieces of bologna tons of lettuce all served between 2 ridiculous pieces of TX toast for $10.50. I guess because they have to pay 8 cooks in the kitchen to make sandwiches.\nAlso, do they really think putting chips on a sandwich is original? \nThis has to be a transplants idea. You can't fool a New Orleanian. Remember, we overstuff sandwiches d...",106
5,tp,"I'm so disappointed with Lebel pizza on 56th & Girard st my food was cold, they delivered half my food & told me I had to pay for all the food including the food they had to bring back to me. Then took an hour to bring my money back and the manager talked to me so rude. This is truly not the type of place I wold recommend with confidence. I'll stick with( Petes on Walnut st.) Sorry Labels this was the third strike.",85
6,tp,"My son just graduated from U of A, so we've been to Caruso's numerous times and always enjoyed the patio and the food is decent if pedestrian. The service has never been great, but last month when we went for my son's graduation the service was TERRIBLE! There were four in our party; three ordered pasta and I ordered an individual pizza. The order took a long time, as usual, and when it came out my pizza was missing. The waitress told me that when it came out the pizza was burnt, so she sent...",328
7,tp,"First of all, who told Marlene's to make breakfast sandwiches taste soooo good. You didn't read that first line correctly. (Clears throat)\n\nAs the late James Brown would shout, ""I feel good!"". Those words are the words that hit me when I bit into the ""Fully Loaded"" sandwich. I nearly shouted, ""It taste good"" in lieu of the words ""I Feel Good!"". When I tell you that the sandwich made me want to kick of my shoes and relax my feet (like the song from my childhood), tha

##### Most confident - true negatives

In [51]:
most_confident_predictions(preds, "tn")

,category,text,num_words
0,tn,Its dim sum is pretty good and its service is satified,11
1,tn,"Got doughnuts here and at rocket 88 for a work meeting. GADC won hands down in the taste, presentation and creativity departments. Delish.",23
2,tn,Had a great time playing trivia and eating yummy food while visiting from Knoxville!!,14
3,tn,Great spot for great music from singer song writers. Heard familiar country music from big stars...but these guys were the writers!! Pretty impressive and entertaining. Every seat is good. A great time!,32
4,tn,Amazing food amazing service AMAZING Shelly and rob are such great people love there food,15
5,tn,I love Blaze because it is super fast and tastes great! Daisa who finished up my pizza made sure I was happy before she boxed it up. Extra arugala for the win!,32
6,tn,First time here with #OperaTastings. The wine selection and food pairings were wonderful accents to the opera arias. We stayed after the private event and enjoyed a bottle of wine and other appetizers. The DJ music was perfect for the atmosphere. I can imagine picking up this place and setting it down in multiple cities across the country.,58
7,tn,Super nice staff and bomb af food I could not recommend this place enough! If you are in to health food the green blast is amazing!,26
8,tn,"Huge portions, Moist Chicken, Very reasonably priced , what's not to love about it!!",14
9,tn,Always good fun here I love the Djs playlist at night hip-hop r&b and classics #90skid,16


##### Most confident - false positives

In [52]:
most_confident_predictions(preds, "fp")

,category,text,num_words
0,fp,They are awesome! They came out in their food truck during early voting at the Perry location and were giving away free slices!! People were waiting in line for several hours to vote and were beyond hungry! Bless them!,39
1,fp,"To the credit of the restaurant - we went on a Sunday afternoon when it was crowded and their power had gone out earlier (?). However... the employee making the sandwiches was moving at a snails pace and showed pretty much no effort to make the sandwiches with care. After the first 30 minutes of waiting, I came inside to see what was going on and proceeded to wait another 20 minutes while I watched him slap together sloppy sandwiches with Mayo and oil all over the outside and ingredients fal...",149
2,fp,"We had a very long wait, the lobster bisque was not very good. I couldn't finish it. I ordered lamb chops, med rare. Waited over 40 min. Chops came out well. Waitress took them back and reordered a filet. It was OK, still over cooked from wed rare. For $35 I had much higher expectations of there dishes. Service was good. Very disappointed for a $100plus meal. Probably won't go back.",71
3,fp,"Arguably one of the worst Dominio's I have ever had the displeasure of using. The pizza is consistently late and cold upon arrival. When you call and inquire why the pizza they said they would deliver in 30 minutes hasn't arrived after waiting two hours, they get snappy with you. Always the same response, ""Your driver just left"". \n\nThen the stupid ""no surprise fees"" campaign. I have been a customer of Domino's for about 20 years. They have raised their delivery fee to $4.99 per order which...",217
4,fp,"Ok. One thing I don't agree with is for a business to advertise the hours of operation & not operate within those times. \n\nThough I understand, consumers generally choose to come into an establishment, like HOOTERS, at the last minute. I get it. However, in the realm of providing exceptional customer service; it is important that any business follow their hours of operation.\n\nLast night, I walked up to this Hooters location and their was a note written on receipt tape that read: ""Last ca...",112
5,fp,"The service , the food and the ambiance combined to deliver a hat trick of Nashville restaurant horror. We are all well travelled foodies who eat out 200+ times per annum. City House has a menu posted that seemed intriguing but when we arrived we found a Sunday supper menu that had virtually nothing enticing. All pastas are removed.. We should have left immediately but on a busy holiday weekend reservations were scarce. The only item we all found we might be able to eat was a meatball sandwi...",150
6,fp,"I'd go back when Tucson ""freezes over"" or after they trained/hired courteous staff. The venue is stunning. The food is good. The service was the worst in town. Our arrogant waiter, Caleb, acted more like a bully on a school ground than a considerate waiter. We had to beg for silverware, got wrong orders that he actually tried to blame on the guests at my table, never were asked if everything was okay or if we needed anything. Clearly, this was because our rude waiter couldn't have cared less.",89
7,fp,The Owner let me sample a large portion of the menu before I decided on a boneless rib sandwich and a half pound of brisket. Everything I tasted was delicious ! Finally a BBQ spot in Wilmington !,38
8,fp,"Great food, terrible environment.\nWe arrived at our reserved time and had to wait twenty minutes to be seated. The restaurant was full and busy. We were compensated with free drinks. OK. Now three of us ordered the tasting menus with eight or so small courses. It took almost three hours to finish. In a high end restaurant this would be fine but not here. The noise level made the waiting between courses painful. We could hardly hear each other talk. The time between dishes was far too long. ...",330
9,fp,"Like any Buffett at this price point, don't expect the world! And like a

##### Most confident - false negatives

In [53]:
most_confident_predictions(preds, "fn")

,category,text,num_words
0,fn,"TIMBERRRRRR\n\n\nGeneric latte. Drive-thru is slow, but staff was nice. Inside is cute.",13
1,fn,Been going here ever since I moved into the area. Lee's Deli is a classic and great to grab a variety of food from. I recommend it to everyone. The owner quality controls everything at the front and I've never had any issues with orders.,45
2,fn,I visit Pattaya a couple times a month for lunch. I have not had anything that I would not try again. The food is very good and Judy is always attention to the needs of the people/table.,37
3,fn,"Fortune cookie is our go-to for Chinese take out on the fly. Consistent quality, fast, friendly service.",17
4,fn,"Such a wonderful place to chill with your kiddos! Visited on Saturday morning with a friend, and both of our kids were so happy to just be able to play and roam around. It's fenced in so my little adventurer can roam free without me worrying. The coffee is fantastic - they had almond milk which always earns brownie points in my book. There was some cute live music playing too which was cool. Just a lovely place and I highly recommend to anyone with kids!",86
5,fn,"Always high expectations when coming here, and they always deliver. Top-notch food flavors drinks and service can't be beat.",19
6,fn,"Interestingly enough I don't enjoy Thai food however my friends do, and we've ordered from Thai Orchid a few times. Quick services and a large menu. The standard staples of pad Thai and green curry came out well. Traditional Thai restaurant - also open for takeout during covid",48
7,fn,So friendly and quick with traditional Chinese food. Try the sizzling rice soup!,13
8,fn,Coming from a Mexican.. Lol this place is legit. They cook the food fresh on the spot for you. Carne asada is a must .,25
9,fn,They have great service here. My family and I always order for pick-up and the food is always ready in a reasonable amount of time and is delicious (especially the Seared Ahi Salad and the Western Crispy Chicken Sandwich). The portions are also a good size. This is my go-to place where I know the food will always be delicious.,60


## Model performance with transfer learning

### Base BERT model, 2 layers finetuned on Bilal

In [54]:
pretraining_model = "bert_base"
finetuning_dataset = "bilal"
n_layers = 2

model_fp = os.path.join(MODEL_DIR, f"{pretraining_model}_{n_layers}_LAYERS_FINETUNED_ON_{finetuning_dataset}", "results")
timestamps = [f for f in os.listdir(model_fp) if f.startswith("2022")]
assert len(timestamps) == 1
results_fp = os.path.join(model_fp, timestamps[0])
preds_path = os.path.join(results_fp, "predictions.csv")
preds = pd.read_csv(preds_path, encoding="utf-8")
test_set = datasets[finetuning_dataset]["test"][["sentence", "label"]]
preds["text"] = test_set["sentence"]
preds["label"] = test_set["label"]
preds["num_words"] = [len(t.split()) for t in preds["text"]]
preds.tail()

,pred_prob_0,pred_prob_1,yhat,y,category,text,label,num_words
1995,0.383555,-0.131563,0,0,tn,Big sale this week. All sort of little gadets that you really don't need but you want them anyways. Prices are really cheap and new stuff brought out all day long. Items for your yard all kinds of tools and hardware holiday decorations and polo shirts and t- shirts. Lots of fun hunting thru boxes and time really fly's fast.,0,60
1996,0.340480,-0.161051,0,0,tn,The new owner and management are great. I didn't tell them I knew anything about cars when I brought mine in to get fixed cause I wanted to see if they would be strait forward and honest with me or try to sell me a bunch of BS I don't need. They were 100% honest about the problem and the price was really good. I'm happy I brought my car here.,0,71
1997,0.297144,-0.015380,0,0,tn,Came here to check out their Patio Furniture. To be honest there was only one choice. But 2 choices in color of the fabric. But it was fine for our backyard. Got a love seat 2 club chairs a coffee table and 2 chaise lounges plus some kind of table. Can't wait for summer. Now I hope the quality is good for the money we spent. Next time it's bar stools if these pieces are of good quality.,0,78
1998,-0.436298,0.305468,1,0,fp,I brought in a flash drive with a 3-page document I wanted printed. I wanted two-sides color 100 copies. The young man behind the counter whom I'll call Skippy said that it would take over an hour. I get it that they may have been busy so I gave them my phone number and left to do other things. 2 1/2 hours later I still hadn't received a call so I head back. I notice that Skippy is gone but a teenager and an older woman are working the printing counter. They're helping other customers both o...,0,223
1999,0.266044,0.006160,0,0,tn,Super helpful. Taught me exactly how to gel stain refinish my curio cabinet. So excited to put it back into my renovated house. This is our go to hardware store for knobs hinges and specialty items. Fairly priced. Worth the trip.,0,41


##### Accuracy

In [55]:
accuracy(preds)

0.6565

##### Classification report

In [56]:
with open(os.path.join(results_fp, "classification_report.txt")) as report_file:
    print(report_file.read())

              precision    recall  f1-score   support

           0       0.63      0.77      0.69      1000
           1       0.70      0.54      0.61      1000

    accuracy                           0.66      2000
   macro avg       0.67      0.66      0.65      2000
weighted avg       0.67      0.66      0.65      2000




##### Confusion matrix - raw & normalized

In [57]:
confusion_matrix(preds, False)

,Pred = False,Pred = True
,,
Actual = False,770,230
Actual = True,457,543


In [58]:
confusion_matrix(preds, True)

,Pred = False,Pred = True
,,
Actual = False,0.3850,0.1150
Actual = True,0.2285,0.2715


##### Word count comparison

In [59]:
preds.groupby(["category"])["num_words"].mean()

category
fn     98.126915
fp    206.078261
tn     69.736364
tp    262.252302
Name: num_words, dtype: float64

##### Most confident - true positives

In [60]:
most_confident_predictions(preds, "tp")

,category,text,num_words
0,tp,But by all means if you find a cheaper price elsewhere please come back and let us know and we'll match it! Yes it's the least I can do after I would have to waste my time elsewhere pricing out everything only to find out you have THE most expensive appliances in the city. We walked around for about 15 minutes completely CLUELESS about which fridge to buy. We had no idea what fridge did what which ones would fit or which would be right for us. There were about 3 customers in the store two o...,411
1,tp,Probably the best $ 10 I've spent. I have oily skin. Which sucks. When I pose for the camera it shines. So I came to Sephora to find a solution. I was intimidated with the oversupply of powerful estrogen and womanhood in the room - zebra style shelves and tons of make ups. Beauty industry must be booming. Shy and embarrassed I approach the sales associate with my dilemma. She paused and contemplated and came up with two possible solutions; a powder or a paper. Because I was and still am unwi...,284
2,tp,How you goin' call us rude? 'Bitch because ya'll ARE! ' 'Good God Cheryl Miranda - you have GOT to quit calling people bitches! ' haha! love it. Okay okay. that DIDN'T really happen. but it would've been SO cool if it did! But what DID happen was. there have been some small changes that make all the difference at this small store and for the most part I really like it. First thing they have added things representative of other tribes to their stock. I have found Fresh Corn (LOVE THEM!) Shir...,408
3,tp,We were looking to sell some of our DVD and CD's that were just sitting around the house. Yes I could use eBay or Craig's List but I wanted an easier way without having to ship things out. Thanks to Yelp we made our way to ZIA! They really made the selling process easy. We probably brought in over 100 DVD's and 100 CD's. It took them a few hours to open check for scratches and scan in. Once they have it all completed they will call you so you wouldn't have to wait around. The payment amount ...,217
4,tp,Had to review this company to give some props especially since reviews are currently failry poor. I have a home warranty. When my pool pump went out these guys were assigned as my repair people. I was liery (being a yelper through and through) because of the negative reviews but couldn't be happier in the end. They called me the same day the claim was assigned to them and were able to come out the next morning (a Friday) between 8 and 12. The guy (Jimmy) rolled up around 9 or so and quickly ...,243
5,tp,Never have I been treated with such utter disregard. Tonight will probably be the last time I set foot in this Target. Since my Mother works at Sunrise Hospital I tend to stop in from time to time . We enjoy the pizza and icees available here and decided to check out Funko Pop Vinyls. While being back in the toy section we opted it to check out Halloween shop. My Five year old son decided he wanted this Ant Man costume and they had a coupon listed you get $10 off if you spend $40 in the Hall...,514
6,tp,Just got back from a Magenta Photo shoot with my niece. Oh boy it was expensive. Here's what I think. If you're doing with a loved one you would enjoy it. It certainly is a bonding experience - the photographer asked us to laugh touch our noses tickle each other play with the bubbles and all other poses. You get to be a celebrity for a day. It was fun. But it put a hole i your pocket. I paid almost $400 for 9 photos of 8x10. My niece screamed when she learned how much those pictures worth an...,227
7,tp,This is a writeup on the pharmacy department. I called in early this morning around 10am to transfer a prescription. I told the associate that I would be in around 3pm once she asked to pick it up. I arrived today around 3pm. I was told it would be twenty minutes. Here starts the irritation. I was on my hour lunch so I figured no biggie I can grab lunch and bring it back to work. It's now 345 and I'm told 

##### Most confident - true negatives

In [61]:
most_confident_predictions(preds, "tn")

,category,text,num_words
0,tn,Une belle place je ne suis pas un acheteur de cossin. Vraiment un bon gift shop mais vraiment plus un truc poussé pour les femmes. Je trouve que c'était bien. Mais un peu dispendieux. J'ai de la misère a payé pour des souvenirs. Je ne comprends l'idée d'avoir des bidules.,50
1,tn,Best Gas prices around except for of course it's competitor Costco which also has great prices. But with that being said - it's always cheaper to gas up @ sams club compared to any other corporately owned gas station. You will easily save 10- 20 cents per gallon. Additionally this sams club gas station also has a automatic car wash which is very affordable! Great product very good price comparatively and usually there is a kinda weird/creepy attendant around if you need help or have any ques...,95
2,tn,Vous cherchez un cadeau original pour la fÃªte de quelqu'un? Farfelus est lÃ pour Ã§a. Plein d'articles aussi 'Farfelus' les uns que les autres. Mon préféré? la fourchette rotative pour manger les spaghettis x). Il y a aussi une variété d'articles de fÃªtes ballons cartes etc. Les vendeurs sont accueillants. Et les prix ben tout dépend de ce que vous voulez prendre =).,63
3,tn,Actually not bad of a place as it is with most thrift store the variety and the availability is really changing. So you can get lucky or not. If you are grossed out by places like 'village des valeurs' and renaissances. It's a great place for you since all the good stuff is already skimmed out. In fact there is a few stores like this one almost neighbouring each other so if you want the full experience. Look around on Saint-Laurent. But to my taste it's a bit lacking this 'thrifty feel'.,92
4,tn,I had a tubeless road bike wheel that was leaking sealant all over. I brought it in to use the compressor thinking a blast of air would seat the tire on the rim better. Eric carefully analyzed the problem took it apart cleaned it all up and remounted it for me. He was so friendly professional and charged me very little for the time and work he put into it. At Landis I find all of the employees to be super thorough and friendly. I am always greeted immediately and someone helps me out right a...,113
5,tn,Contente que les magasins ecolo et bio deviennent de plus en plus accessibles bouffe epices en vrac savons en vrac etc.,21
6,tn,Great place to shop at if you don't mind valet or parking at their parking garage behind the mall. Enjoys all the little shops it might be a bit hot during summer time but with the misters going on you'll be just fine. This mall sits on a very busy major intersections at 24th St. & Camelback Rd. Very classy neighborhood with a lot of stores and shops and if you don't mind paying for valets definitely shop here.,79
7,tn,This store carries a wide variety of very high quality canvas bags if you are willing to pay up. Granted it took me 3 tries to find this place but once you do and not fail like I did you will be enticed to purchase goodies. Wonderful camera gear. Bought a DSLR camera strap here and no more neck soreness from the gigantic telephoto.,64
8,tn,TJ took care of us with laptop update. great customer service. Had to go back and get some things done no wait and he was on it quick as always. Thanks TJ.,32
9,tn,Hell of a lot better variety and prices then Lowes. Lowes is over priced and there is never anyone to help you and I never find what I need compared to when I goto home depot. Home depot always has someone around to help. They are always my first choice when I need stuff.,54


##### Most confident - false positives

In [62]:
most_confident_predictions(preds, "fp")

,category,text,num_words
0,fp,Visited the store on the date of this post at about 5 PM. If I hadn't of fallen in love with the hat I picked up I would have fled the store. The line--holy cracker--felt like the DMV. There were only two cashiers and then a floor supervisor/manager/head cashier/top coat factory worker kept wandering around the line and telling customers: 'that is so cute. I want to buy that. ' Or 'Look at your son! He is so cute. Look. He's cute. Look. ' Come on lady. Open up another register! I don't want ...,360
1,fp,A friend told me about Ulta. My daughter and I stopped in for a look. It was my first visit to such a place. Wow. If you enjoy lots of stuff to pamper and preen with this is the place for you. Not only do they have the take-home goods there is also an in-store hair salon. My daughter -- who has become such a PRODUCT junkie -- was in her element. All the makeup nail polish hair products etc. I liked the make up and perfume sections. We stayed way too long but thankfully I escaped without spen...,298
2,fp,I placed a special order for a board game that they didn't have in their online shop the guys were cool enough to order it for me and priced it very well. Wood for sheep has become my go to online retailer because they offer a free pickup option at any of the local events they attend and I like to save on shipping as much as possible! I see them all the time at events like GameOn! And TABScon. Hearing that they just opened their new Game Centre I just had to stop by on a quiet weekday aftern...,225
3,fp,I feel bad for not even thinking to write a review for this store sooner. However I selfishly didn't really want to add yet another great review as I'm afraid you'll go buy out my size before I get there ha ha. But they deserve more biz - I want them to be around a long long time! So get in there and buy a lot like I do. I fell in love with it when I walked by the window 2 years ago & had to have one of the cutest cover-ups I've ever seen. I was terrified to find out the price since I was at...,306
4,fp,My previous review was kind of harsh. Honest and true and not mean-spirited but harsh. But sometimes just sometimes things change for the better. Luckily for Summer Winds (and for me) this is one of those times. Let the harshness be gone! As of late they've turned things completely around from what I've become accustomed to. The awesome friendly knowledgeable staff are still there. But now they have awesome living and completely weird plants to go with them. I have never seen so many varieti...,210
5,fp,This review is for the casino only as I didn't stay here but at The Wynn next door. It was Friday night. We were supposed to go to the Marquee nightclub to see some DJ we had tickets for but we had gone to Tao the night before and were not in the mood. We went to Encore casino around 4 am. Besides all the drunk people leaving the nightclubs the casino itself was fairly dead. We played slots. A cute cocktail waitress came up and asked us if we wanted a drink Yes please! I ordered a margarita....,222
6,fp,The two stars given to this particular (3rd) review of mine is only because of this one experience. Normally I go in everyone's friendly I get what I need and I leave happy. albeit about $20-40 lighter. Well since I've been noticing how much I spend here I finally remembered to bring along one of the coupons they send out every week. It was a $3 off when spending $30 or more. Seems easy enough right? Well. So I wandered around not finding much that I needed but the boyfriend had a basket ful...,261
7,fp,The Sixth Annual Trailer Trash White Elephant post-Christmas party was once again catapulted into the stratosphere thanks in part to thrift store shopping the day of. Fortunately I'm house sitting in the area and was able to swing by here after breakfast Saturday morning. The huge sign in the window beckoned: 50% off Saturday Dec. 26! Score. My first stop on this treasure hunt was a Jello mold for a dess

##### Most confident - false negatives

In [63]:
most_confident_predictions(preds, "fn")

,category,text,num_words
0,fn,This place is an authorized dealer but seem like a Verizon Wireless corporate store from their signage. Staff is friendly but charge for services that are free if you go to a corporate store so if you want to save money go to a corporate store.,46
1,fn,I was just here for the roller derby finals so all I know is that this is an awesome arena for roller derby! Great big area in the entry for vendors bar snacks. Bummer that the restaurant is a Subway instead of something local though! There were all kinds of other rooms and areas for ju jitsu and a track for running and every seat in the area seemed to offer a pretty great view of the action. The location is great near the Olympic Stadium and Parc Maissoneuve and tons of parking for both bik...,98
2,fn,Il y a plusieurs boutiques de friperies sur la Main mais la première Ã avoir ouvert c'est celle-ci. Et c'est aussi la seule qui offre une sélection soigneusement recherchée. Le gérant n'achète pas tout et n'importe quoi en vrac dans le seul but de se donner un air vintage il choisit avec soin chacune des pièces qu'il propose ensuite Ã ses clients. Alors c'est sÃ»r tout Ã§a a un prix. Si on est prÃªt Ã le mettre on vient ici sinon on va plonger dans le bac Ã fripes Ã 1$ chez Eva B.,94
3,fn,First time in I had a vamo with me that didnt work and was thinking they could help me. I was WRONG! I asked for help walking up and they (females) said how the vamo sucks and they cant stand it. I went for help and received shitty attitude about my mod I left fast after that real fast JUICE SUCKS SERVICE SUCKS ATTITUDE SUCKS not worth my time! you can get anything they can. just look it up! never give up and bendover too the words of the car salesman think about it.,94
4,fn,A lot of people think the guy that runs this place is a total cock. I can see that. If you're not in there asking for the latest in indie rock or hip hop this guy might sneer a bit. However I like him. I get off playing the 'Name an Obscure Band that Pitchfork Really Likes' game with him. Plus he's always given me great recommendations. I've picked up special orders and hard-to-find indie stuff here and I'll keep going back sneer-be-damned.,83
5,fn,Really great group of people that actually care about their customers and the bikes they sell/fix. It's a great place to get your bike fixed up and they won't charge you an arm and a leg. Also great place to get used parts if you don't want to replace a broken part with a brand new one that costs more than your whole bike altogether. He will also take trade ins but won't buy bikes so you know you're not buying a stolen bike if you buy one of their refurbished used bikes.,93
6,fn,Nice shopping experience I have spent many hours at Tanger Outlets. Tanger offers many great shopping options. There's Nike Under Armour Puma Sacks 5th Banana Republic Ralph Lauren and so many more. You have to search in order to find the good 'discount deals' but the deals are there for the taking if you have the time to look. The only drawback is you will need to go to Westgate to eat because there aren'r any restaurants at Tanger unless you want pizza.,83
7,fn,While looking at replacing some toilets we stopped by Lowe's to check the prices. The question I asked the chipper employer while pointing at a specific toilet was 'Is this the only one piece toilet you have? ' She responded 'No. We have lots of toilets' as she pointed to the row of two piece toilets. I repeated my question 'Is this the only one piece toilet you have? ' Her response 'Do you want a one piece toilet? ' No genius I'm asking for my health. This left me super frustrated so we left.,95
8,fn,Ridiculous policy of not calling dealer to make an offer on their merchandise. Dealer #92 had random pieces of China being called a set. Personnel refused to contact dealer. If this is truly your policy you deserve to be out of business like the rest of the antique stores in the area. Shame on you. Total disservice to you

### Base BERT model, 4 layers finetuned on Bilal

In [64]:
pretraining_model = "bert_base"
finetuning_dataset = "bilal"
n_layers = 4

model_fp = os.path.join(MODEL_DIR, f"{pretraining_model}_{n_layers}_LAYERS_FINETUNED_ON_{finetuning_dataset}", "results")
timestamps = [f for f in os.listdir(model_fp) if f.startswith("2022")]
assert len(timestamps) == 1
results_fp = os.path.join(model_fp, timestamps[0])
preds_path = os.path.join(results_fp, "predictions.csv")
preds = pd.read_csv(preds_path, encoding="utf-8")
test_set = datasets[finetuning_dataset]["test"][["sentence", "label"]]
preds["text"] = test_set["sentence"]
preds["label"] = test_set["label"]
preds["num_words"] = [len(t.split()) for t in preds["text"]]
preds.tail()

,pred_prob_0,pred_prob_1,yhat,y,category,text,label,num_words
1995,0.117500,-0.616239,0,0,tn,Big sale this week. All sort of little gadets that you really don't need but you want them anyways. Prices are really cheap and new stuff brought out all day long. Items for your yard all kinds of tools and hardware holiday decorations and polo shirts and t- shirts. Lots of fun hunting thru boxes and time really fly's fast.,0,60
1996,0.411693,-0.818778,0,0,tn,The new owner and management are great. I didn't tell them I knew anything about cars when I brought mine in to get fixed cause I wanted to see if they would be strait forward and honest with me or try to sell me a bunch of BS I don't need. They were 100% honest about the problem and the price was really good. I'm happy I brought my car here.,0,71
1997,-0.013836,-0.480445,0,0,tn,Came here to check out their Patio Furniture. To be honest there was only one choice. But 2 choices in color of the fabric. But it was fine for our backyard. Got a love seat 2 club chairs a coffee table and 2 chaise lounges plus some kind of table. Can't wait for summer. Now I hope the quality is good for the money we spent. Next time it's bar stools if these pieces are of good quality.,0,78
1998,-0.642779,0.622445,1,0,fp,I brought in a flash drive with a 3-page document I wanted printed. I wanted two-sides color 100 copies. The young man behind the counter whom I'll call Skippy said that it would take over an hour. I get it that they may have been busy so I gave them my phone number and left to do other things. 2 1/2 hours later I still hadn't received a call so I head back. I notice that Skippy is gone but a teenager and an older woman are working the printing counter. They're helping other customers both o...,0,223
1999,0.218352,-0.761208,0,0,tn,Super helpful. Taught me exactly how to gel stain refinish my curio cabinet. So excited to put it back into my renovated house. This is our go to hardware store for knobs hinges and specialty items. Fairly priced. Worth the trip.,0,41


##### Accuracy

In [65]:
accuracy(preds)

0.696

##### Classification report

In [66]:
with open(os.path.join(results_fp, "classification_report.txt")) as report_file:
    print(report_file.read())

              precision    recall  f1-score   support

           0       0.70      0.70      0.70      1000
           1       0.70      0.69      0.70      1000

    accuracy                           0.70      2000
   macro avg       0.70      0.70      0.70      2000
weighted avg       0.70      0.70      0.70      2000




##### Confusion matrix - raw & normalized

In [67]:
confusion_matrix(preds, False)

,Pred = False,Pred = True
,,
Actual = False,697,303
Actual = True,305,695


In [68]:
confusion_matrix(preds, True)

,Pred = False,Pred = True
,,
Actual = False,0.3485,0.1515
Actual = True,0.1525,0.3475


##### Word count comparison

In [69]:
preds.groupby(["category"])["num_words"].mean()

category
fn     77.901639
fp    191.052805
tn     61.988522
tp    235.233094
Name: num_words, dtype: float64

##### Most confident - true positives

In [70]:
most_confident_predictions(preds, "tp")

,category,text,num_words
0,tp,The Noob's guide to a $20 massage at Bali! If you guys reading this are like me you would consider yourself very INEXPERIENCED when it comes to massages in general. And of course you've heard plenty of stories about the shady Asian Massage parlors in Vegas LOL. Since I am now an expert (jk only been one time lol) I will HELP you out and let you know what to expect! 1) I showed up with no appointment but was promptly welcomed and walked to the back. Maybe during prime time you might have to w...,733
1,tp,Now where is that six star button? Honestly a couple months ago my husband and I spent an entire day with the owner Johanna literally designing molding and creating each other's wedding bands from scratch. Yes you heard me right. We MADE each other's wedding bands. Three words: Best. Date. Ever. We chose to make ours in sterling silver (I'm allergic to gold) but you can also make gold rings as well. In a matter of 5 hours we turned a couple beads of silver into two gorgeous wedding bands. Jo...,278
2,tp,How you goin' call us rude? 'Bitch because ya'll ARE! ' 'Good God Cheryl Miranda - you have GOT to quit calling people bitches! ' haha! love it. Okay okay. that DIDN'T really happen. but it would've been SO cool if it did! But what DID happen was. there have been some small changes that make all the difference at this small store and for the most part I really like it. First thing they have added things representative of other tribes to their stock. I have found Fresh Corn (LOVE THEM!) Shir...,408
3,tp,Look. Don't go anywhere else to buy fine jewelry ok? You hear me? I've known the owners of this place since 2005. They are the most charitable couple I've ever met. Now. You may not care about charity. Ok. That's cool. That's your right. So what if you don't. But do you care about rocks? Some big and shiny others smooth and colorful mostly sparkly and attractive yet many rare and unique. ROCKS? Treasures is owned by an awesome family! . THEY ROCK! (Ha!) . Their employees rock too! (Ha HA!) ....,440
4,tp,Time lost. An ok place to buy you may find something that you may like but this review is mostly about their buying department. I usually donate my used clothes at the local salvation army but still always end up with some clothes through the years that I just can't part from. You know those favorite shirts shoes or pants that you once wore to that special place or event back in the days and are hoping to wear them again someday but you never do? Today against the wishes of the other me deci...,402
5,tp,its like clockwork Rockport -Wu Tang Diesel The validation of Rockport shoes in a Wu Tang track indicates that Rockports aren't just for old men and spiffy executives who wear their slacks above waistline looking for 'weekend casual wear'. Rockport is the real deal and I'll take it over Timberlands any day. It all began in boxing day when the Rockport shoe store in Chinook held a 50% off sale. This allowed me to afford a pair of their boots that was normally priced at $250+. These boots are ...,274
6,tp,When does an installation become NOT an installation? Here's how I think this works: You go to a Big Box store see all those appliances those plumbing fixtures good prices and. hey wait a minute! They'll even install this for me for what looks like a great price? Sign me up Orange Apron Guy. So they hire some OTHER company to do the install. That's pretty standard and a nice way to off-shore the responsibility in the parlance of our times. That Other Company then schedules to come out to you...,460
7,tp,WOW. This is the FIRST place in Vegas and Henderson that I have ever come across any vintage finds with GREAT prices I was ecstatic! The store was well organized the staff was super friendly and helpful. Finds: 70s/80s romper ($7) 50s style skirt ($1) 50s style skirt ($4) 80s dress ($4) Catherine Malandrino dress ($4) and a 70s blouse ($1)-- AHH I just about lost my MIND! LOVE IT. Another thing that I lov

##### Most confident - true negatives

In [71]:
most_confident_predictions(preds, "tn")

,category,text,num_words
0,tn,Always fast and courteous.,4
1,tn,Merchandise sales and customer service has taken a turn for the worst after relocating.,14
2,tn,You have to go just to see the window art.,10
3,tn,Amazing vibes original products. Fair pricing. I have only been in once but will defiantly return.,16
4,tn,My client bought a new wallet & the sales lady was very kind & helpful w/ him.,17
5,tn,It isn't there anymore.,4
6,tn,Busy store friendly staff huge inventory and awesome products. This store is in a heavily populated area and it can be annoying getting in and out so beware.,28
7,tn,nothing special but im excited for a new urban outfitters because the mandalay bay location is too far.,18
8,tn,Very unprofessional when dealing with patients really people in general who are in need of the help from the pharmacy. Now I have been disappointed multiple times- reason why I have to mention it on the internet.,37
9,tn,Great customer service and wonderful selection for things I need for school! This store is full of good quality and colorful and products for all things school and office.,29


##### Most confident - false positives

In [72]:
most_confident_predictions(preds, "fp")

,category,text,num_words
0,fp,Frank Gehry You're a genius! --Frank Gehry the Simpsons The ARCHITECTURE: Seriously this is one of my favorite buildings that Frank Gehry has done. It's functional a really cool look and the design serves the space rather than the other way around. The cool super-skeleton admits a lot of light. The looping staircases are a nice architectural element that enhances the space. The ART: There are a few rather strong collections here most notably the work of the Group of Seven artists. There's a...,195
1,fp,About three years ago Judy aand I met Nancy Doney at the Nordstrom's in the Hosiery Department in Chandler Fashion Center. We like shopping there for a variety of Hue jeggings. Nancy was always warm welcoming to us genuinely happy to see us and above all extremely happy to assist us in any way she could. Now Nancy works at Nordstrom Rack on Chandler Boulevard where she continues to exceed by helping customers any way that she can. Nancy's customer service excels that of anyone I've ever met ...,273
2,fp,Desperate times - a dead coffee maker. Call for desperate measures - shopping at Kohl's. The fact I dislike this store makes me an anomally. Some of my best friends and a good portion of the free world shop at Kohl's. What burns me up is shopping while looking over my shoulder! What am I looking for? The next sale the 'mother' of all sales to drop! And darn if it didn't happen on my last visit! I found a coffee maker 'on sale' at a great price arrived happily at the store was ecstatic to see...,237
3,fp,I feel bad for not even thinking to write a review for this store sooner. However I selfishly didn't really want to add yet another great review as I'm afraid you'll go buy out my size before I get there ha ha. But they deserve more biz - I want them to be around a long long time! So get in there and buy a lot like I do. I fell in love with it when I walked by the window 2 years ago & had to have one of the cutest cover-ups I've ever seen. I was terrified to find out the price since I was at...,306
4,fp,(I have only been to one exhibit here so until I see how other exhibits fare 3 stars. Also this is a piece I wrote for a magazine that never published it. ) 'People either love Andy Warhol or they hate him' quipped Warhol Out West exhibit tour guide Gina 'And I think he loved that people felt that way about him. ' It's easy to see why people feel so strongly. In today's world of Instagram filters and Photoshop people may look at his work and say 'I can do that! ' or 'that's not art! ' But do...,439
5,fp,If I could give negative stars I would do so . I will be contacting the better business bureau as well to file a complaint. Let's go back to sept 2014 I was excited to our beautiful custom front entry double doors in a rustic design for my beautiful scottsdale home. I'm told doors will arrive by late October no problem or so I thought . I receive a call early November stating my doors were in so my contractor picked them up and took out my old doors. First problem to arise was they were pre ...,247
6,fp,Unreal. Lon in Electrical is amazingly knowledgeable. He is a great listener and helped me pick out the perfect items. Everything you hope for when you walk in with a vague idea and zero skill set. I hate home improvement because it always means I end up making 3 trips. Lon makes me look forward to needing his help on a new project! Sarah in paint was also a Rock Star. Thanks! A year ago Lowes got a five star review see about. Today they get a 1.If there were negative numbers they would be s...,435
7,fp,This is the kind of experience that you could rate anywhere from 1 star to 5 stars. Why? Because its mad crazy! It's a freak show. it's tacky. It's got live bands. it's got talented artists impersonators show girls Elvii of all kinds tourists galore a laser light show that ranges from weird to awesome overpriced objects for sale lots and lots of casinos and eateries. I mean you have to give it 5 stars bec

##### Most confident - false negatives

In [73]:
most_confident_predictions(preds, "fn")

,category,text,num_words
0,fn,i love reid's for the stationary ribbons wrapping paper and moleskines (what? you don't have to buy them at chapters? who knew). this store is also an excellent place to find gifts or sarcastic/inappropriate greeting cards.,36
1,fn,Roses have always been fresh and beautiful! Arrangements are pretty and delivery is always on time.,16
2,fn,This Bath & Body Works is never crowded but that's because Fiesta mall is practically empty (and was recently sold.,20
3,fn,I love this Costco I work nearby and stop here for lunch on a regular basis. I also buy some household items in bulk here. I prefer this location versus the one in Middleton which is smaller. I would recommend avoiding this place on Friday's Saturdays and Sunday's.,48
4,fn,For some background please see my review for the Aldi on Hwy 160.,13
5,fn,This just opened in my 'hood! YAY! It took over the historical corner space formerly occupied by Noah's Ark.,19
6,fn,On the occasion that I needed some help the staff went out of their way to find what I needed. I am impressed with the diversity of merchandise at this location as well.,33
7,fn,They have products most stores don't have. Sometimes you can get the same products on E-Bay for the same price. Never listen to advice from anyone in this store though. Ever.,31
8,fn,I shop here every year for Christmas gifts for folks back east where authentic Native American jewelry is hard to come by. Alway a great experience everyone is super friendly in here and I enjoy coming here a few times a year. Great stuff.,44
9,fn,So i had to find a new outfit for the hubbys birthday which we celebrated at TAO A few weeks ago and i found my gorgeous dress here! Wow upscale personal service in a trendy affordable boutique with lots of fabulous finds! I need to come back.,47


### Amazon small model, 2 layers finetuned on Bilal

In [74]:
pretraining_model = "amazon_finetune"  # Meaning "small"
finetuning_dataset = "bilal"
n_layers = 2

model_fp = os.path.join(MODEL_DIR, f"{pretraining_model}_{n_layers}_LAYERS_FINETUNED_ON_{finetuning_dataset}", "results")
timestamps = [f for f in os.listdir(model_fp) if f.startswith("2022")]
assert len(timestamps) == 1
results_fp = os.path.join(model_fp, timestamps[0])
preds_path = os.path.join(results_fp, "predictions.csv")
preds = pd.read_csv(preds_path, encoding="utf-8")
test_set = datasets[finetuning_dataset]["test"][["sentence", "label"]]
preds["text"] = test_set["sentence"]
preds["label"] = test_set["label"]
preds["num_words"] = [len(t.split()) for t in preds["text"]]
preds.tail()

,pred_prob_0,pred_prob_1,yhat,y,category,text,label,num_words
1995,0.297750,-0.306897,0,0,tn,Big sale this week. All sort of little gadets that you really don't need but you want them anyways. Prices are really cheap and new stuff brought out all day long. Items for your yard all kinds of tools and hardware holiday decorations and polo shirts and t- shirts. Lots of fun hunting thru boxes and time really fly's fast.,0,60
1996,0.091341,-0.406960,0,0,tn,The new owner and management are great. I didn't tell them I knew anything about cars when I brought mine in to get fixed cause I wanted to see if they would be strait forward and honest with me or try to sell me a bunch of BS I don't need. They were 100% honest about the problem and the price was really good. I'm happy I brought my car here.,0,71
1997,-0.017601,-0.022443,0,0,tn,Came here to check out their Patio Furniture. To be honest there was only one choice. But 2 choices in color of the fabric. But it was fine for our backyard. Got a love seat 2 club chairs a coffee table and 2 chaise lounges plus some kind of table. Can't wait for summer. Now I hope the quality is good for the money we spent. Next time it's bar stools if these pieces are of good quality.,0,78
1998,0.026037,0.252907,1,0,fp,I brought in a flash drive with a 3-page document I wanted printed. I wanted two-sides color 100 copies. The young man behind the counter whom I'll call Skippy said that it would take over an hour. I get it that they may have been busy so I gave them my phone number and left to do other things. 2 1/2 hours later I still hadn't received a call so I head back. I notice that Skippy is gone but a teenager and an older woman are working the printing counter. They're helping other customers both o...,0,223
1999,0.043290,-0.470180,0,0,tn,Super helpful. Taught me exactly how to gel stain refinish my curio cabinet. So excited to put it back into my renovated house. This is our go to hardware store for knobs hinges and specialty items. Fairly priced. Worth the trip.,0,41


##### Accuracy

In [75]:
accuracy(preds)

0.6255

##### Classification report

In [76]:
with open(os.path.join(results_fp, "classification_report.txt")) as report_file:
    print(report_file.read())

              precision    recall  f1-score   support

           0       0.64      0.59      0.61      1000
           1       0.62      0.66      0.64      1000

    accuracy                           0.63      2000
   macro avg       0.63      0.63      0.63      2000
weighted avg       0.63      0.63      0.63      2000




##### Confusion matrix - raw & normalized

In [77]:
confusion_matrix(preds, False)

,Pred = False,Pred = True
,,
Actual = False,590,410
Actual = True,339,661


In [78]:
confusion_matrix(preds, True)

,Pred = False,Pred = True
,,
Actual = False,0.2950,0.2050
Actual = True,0.1695,0.3305


##### Word count comparison

In [79]:
preds.groupby(["category"])["num_words"].mean()

category
fn    132.734513
fp    141.536585
tn     72.991525
tp    215.204236
Name: num_words, dtype: float64

##### Most confident - true positives

In [80]:
most_confident_predictions(preds, "tp")

,category,text,num_words
0,tp,Won't even try to deny it. I'm a Bath & Body Works junkie! There's a Wallflower in almost every room of my house; hand soap in my bathrooms and kitchen. If B&BW ever discontinued the Sensual Amber Forever Red or Country Chic body care lines I might curl up and die- been using Sensual Amber since 2007! Yeah you get the picture. Associates are always super helpful and fun to chat with whether you're looking for something new to try getting your standbys or simply browsing on your lunch break. ...,207
1,tp,A bastion. A hold-out. A sanctuary. With global homogenization of nearly everything musical a small independent and locally-owned guitar store is a rare duck indeed. Yet here it is Timeless Guitars railing against the Guitar Centaur defiant in the face of the mindless mewling masses and their meandering march to internet mass-merch. What a glorious store. Walls full of odd obscure guitars basses amps. The back room has a bunch of acoustic guitars too but their strong suit is the 'starter gui...,174
2,tp,Today 3/12/2013 I checked out the new Caswells Shooting Range in Tempe-it's across from IKEA. The correct name for this location is C2 TACTICAL-really? I am not a would be soldier of fortune or someone pretending to be a Seal or Green Beret so the name does not impress me. F.Y.I- I was caught in a very bad draft from 1967-1969. Maybe some shooters are just watching too many Rambo or Police/Action movies? That could explain some of the silly actions witnessed at any shooting range. For months...,248
3,tp,I am never coming here again. I don't care if they offer me free oil changes for life with bubbling crude straight out of Jed Clampett's oil well. I think the staff at this place were trained by Bernie Madoff or something because I got a full-on professional fleece job at this place. I needed an oil change before I headed out to DC so I took my car here because I've had good experiences with the NTB in Westlake. I checked in and the manager practically raced outside to do an inspection. He c...,399
4,tp,I wish I could give 2 stars for the selection and pricing and 4 stars for the fitting experience but I guess I'll just have to settle on 3. LIKES 1. The bra fitting was a great experience. I made a reservation for me and my mom about a week beforehand for a personal fitting. I learned a lot of valuable things about how a bra should fit bra types like contoured vs. seamed vs. demi and the good & bad about each. Also I discovered how proper bra fit was important for breast health. 2. I was abl...,475
5,tp,The best way to describe CSA short and sweet? NO PRESSURE! I've been to all the different shops/ranges in the area and in the Charlotte area you have a lot of choices. Carolina Sporting Arms has something for every gun aficionado! When you walk in you're greeted right at the front door friendly staff will point you in the right direction whether you're heading to the range or bringing a firearm in for gunsmith work. Everyone here works as one well-trained unit to ensure their customers are s...,376
6,tp,Here we go again. I have an android phone (all screen already. ) and iPhone. I wanted to try the all screen iPhone X. I got an offer I couldn't resist on my current iPhone. Walked in. Look at the X and waited to see if I can find someone. 15mins later and getting hot under the collar (from the wait and the fact that I have a jacket on - it is cold out you know) I went to see an Apple Helper. OMG - I broke the Golden Rule at Apple. I walked past the Orchard Keeper - the one that supposedly gr...,393
7,tp,Came here on a recommendation on where to pick up ammo. I usually went to either Walmart or Sportsman's Warehouse but after coming here VM is now my go to spot for ammo. What's great about this place is that its centrally located located off Russell rd a few minutes driving off the Vegas strip. They carry a wide range of calibers of ammo for all types of use including JSP and JHP for self defense and FMJ

##### Most confident - true negatives

In [81]:
most_confident_predictions(preds, "tn")

,category,text,num_words
0,tn,What a great place. I need to buy one of the CLE winter hats one of these days! I hope this place does well.,24
1,tn,This is a great location! Great variety and always have a sale section :) Working are friendly and knowlegable.,19
2,tn,One of my favorite record stores in Phoenix. This is what it use to be like back in the 70's and 80's kiddies. Great staff great selection GREAT STORE.,29
3,tn,Always fast and courteous.,4
4,tn,my husband could get lost in here for days! my daughter was in love with a particular gun! and we found several items for Father's Day.,26
5,tn,We love this place and the staff! Mick and Kat rule. We bring our dog here! Super friendly people and fun shopping environment. I like their selection of boards shoes and accessories. We stop by once a week.,38
6,tn,Really. I have no idea how much money my wife and I have spent here. We bought whole lot of kids cloth for our nephews. This place is very convenient to us which there are many stores nearby. The staff is always very friendly. They don't mind go through their inventor and told us there are some new items in the back. So we are very happy to shop at this place.,72
7,tn,Love BMV but I wish their Latin American history/politics section consisted of more than Che and Fidel bios.,18
8,tn,Like like like Dollar Tree! Just as nearly always went home with more than I went for.,17
9,tn,This place has a lot to offer. They have a lot of events and things to bring in the people and they are active in the community. It's a lot of fun to see what all is going on in here. Definitely one to see for yourself.,47


##### Most confident - false positives

In [82]:
most_confident_predictions(preds, "fp")

,category,text,num_words
0,fp,If I could give negative stars I would do so . I will be contacting the better business bureau as well to file a complaint. Let's go back to sept 2014 I was excited to our beautiful custom front entry double doors in a rustic design for my beautiful scottsdale home. I'm told doors will arrive by late October no problem or so I thought . I receive a call early November stating my doors were in so my contractor picked them up and took out my old doors. First problem to arise was they were pre ...,247
1,fp,I went to the printing department for documents for my business. The initial young man that helped was very helpful and personable. He was from a different store and not super familiar with this department. I printed out one copy for a presentation and wanted to see it live and in real time in case changes needed to be made. We sat down to review said document and discuss any changes. The line quickly became 5 people deep with lots of other employees milling about. This guy didn't not get th...,307
2,fp,This store's customer service is horrible. I went in one day for a free hairstyle with living proof vendors which they did an amazing job. However I wanted to try on some makeup products and needed help trying some on. I had a lot of money to spend that day too. Anyways so I asked one girl working with light purple and gray hair for help. She said 'oh I'm actually gonna help them over there. ' I asked one worked with glasses who worked in the back she told me she was only available through a...,347
3,fp,This review is for the casino only as I didn't stay here but at The Wynn next door. It was Friday night. We were supposed to go to the Marquee nightclub to see some DJ we had tickets for but we had gone to Tao the night before and were not in the mood. We went to Encore casino around 4 am. Besides all the drunk people leaving the nightclubs the casino itself was fairly dead. We played slots. A cute cocktail waitress came up and asked us if we wanted a drink Yes please! I ordered a margarita....,222
4,fp,After my disastrous experience with trying to buy a used computer from ASU (only about $500 for something that can't be used) anything but the word brand new can upset me. I went to this apple store after going to a competitor and can say they really helped me save money. I went in ready to buy a new unit when one young man said why not buy refurbished on line and have it delivered. His logic waas that they had gone farther through a refurbished machine when they got it back than with the re...,412
5,fp,(I have only been to one exhibit here so until I see how other exhibits fare 3 stars. Also this is a piece I wrote for a magazine that never published it. ) 'People either love Andy Warhol or they hate him' quipped Warhol Out West exhibit tour guide Gina 'And I think he loved that people felt that way about him. ' It's easy to see why people feel so strongly. In today's world of Instagram filters and Photoshop people may look at his work and say 'I can do that! ' or 'that's not art! ' But do...,439
6,fp,Stopped in here 1 hour before it closed. Did the body scan that the trainer recommended. Wow! Now it is in black and white. Anyway. the store is locally owned. The young girl inside had me wondering how much she knew but she knows her stuff. She did the test and let us know what it meant. She made me feel better by letting me know females are different and the machine does not differentiate butt and boobs! That is wisdom from this young kid. We shopped around because we needed protein powder...,214
7,fp,Updating My Review: 5 Stars remains! A few weeks ago my cell phone charger that Terrance replaced a year or so ago stopped working again. I happen to have a day off during the week and stopped in. Terrance remembered me and was willing to make an appointment with me for later in the week. When I explained to him that I was willing to wait all day today for him to squeeze me in he said you know what 'I ha

##### Most confident - false negatives

In [83]:
most_confident_predictions(preds, "fn")

,category,text,num_words
0,fn,Love coming here drunk or sober. Every year for my birthday I force someone to play Buck Hunt with me. Fun place to kill some time and quarters.,28
1,fn,Called at 8 am for a estimate was given estimate over the phone and scheduled the spring repair the same day after work. Showed up on time and work was completed in a timely. Would recommend.,36
2,fn,I shop here every year for Christmas gifts for folks back east where authentic Native American jewelry is hard to come by. Alway a great experience everyone is super friendly in here and I enjoy coming here a few times a year. Great stuff.,44
3,fn,good mall for walking. there is a costco movie theater and decent food court.,14
4,fn,Oooooh yeah! A whole lotta variety of guns at this place. However I'm still researching and looking at options. Very knowledgeable professional and courteous staff.,25
5,fn,Creative and cheap decorations always gets me here! What keeps me away the lack of workers which keeps the check out line longer than my actual shopping trip. Before spending hoards of money on basic essentials like soap toothbrushes rubber bands dishes birthday and party supplies simple toys snacks or any other small knick knacks for basket stuffers you should head here. I put together a gift basket for under $10 dollars and it included the basket itself. And every week there is something n...,92
6,fn,Ridiculously expensive but great quality luggage. Had to get a specific piece for my boss and when I called to check availability I was able to do everything via phone and email. Saved me a trip to caesars AND no sales tax since it was shipped out of state. Yay! Manny and Di made it a super easy transaction.,59
7,fn,So i had to find a new outfit for the hubbys birthday which we celebrated at TAO A few weeks ago and i found my gorgeous dress here! Wow upscale personal service in a trendy affordable boutique with lots of fabulous finds! I need to come back.,47
8,fn,Have stayed here a couple of times over the years. At the time I thought Bally's was quite nice. Nowadays I do not. For the price of the rooms I would not recommend staying here. There are much nicer places on the Las Vegas Strip to spend the night at for the same price or lower. What I do like about Bally's is the location. Their location on the Strip is walking distance to all the major hotels and casinos. Will still drop by here if we are taking a stroll of the Las Vegas Strip.,96
9,fn,I love this Costco I work nearby and stop here for lunch on a regular basis. I also buy some household items in bulk here. I prefer this location versus the one in Middleton which is smaller. I would recommend avoiding this place on Friday's Saturdays and Sunday's.,48


### Amazon small model, 4 layers finetuned on Bilal

In [84]:
pretraining_model = "amazon_finetune"  # Meaning "small"
finetuning_dataset = "bilal"
n_layers = 4

model_fp = os.path.join(MODEL_DIR, f"{pretraining_model}_{n_layers}_LAYERS_FINETUNED_ON_{finetuning_dataset}", "results")
timestamps = [f for f in os.listdir(model_fp) if f.startswith("2022")]
assert len(timestamps) == 1
results_fp = os.path.join(model_fp, timestamps[0])
preds_path = os.path.join(results_fp, "predictions.csv")
preds = pd.read_csv(preds_path, encoding="utf-8")
test_set = datasets[finetuning_dataset]["test"][["sentence", "label"]]
preds["text"] = test_set["sentence"]
preds["label"] = test_set["label"]
preds["num_words"] = [len(t.split()) for t in preds["text"]]
preds.tail()

,pred_prob_0,pred_prob_1,yhat,y,category,text,label,num_words
1995,0.516459,-0.537818,0,0,tn,Big sale this week. All sort of little gadets that you really don't need but you want them anyways. Prices are really cheap and new stuff brought out all day long. Items for your yard all kinds of tools and hardware holiday decorations and polo shirts and t- shirts. Lots of fun hunting thru boxes and time really fly's fast.,0,60
1996,0.229717,-0.549590,0,0,tn,The new owner and management are great. I didn't tell them I knew anything about cars when I brought mine in to get fixed cause I wanted to see if they would be strait forward and honest with me or try to sell me a bunch of BS I don't need. They were 100% honest about the problem and the price was really good. I'm happy I brought my car here.,0,71
1997,0.248583,-0.283639,0,0,tn,Came here to check out their Patio Furniture. To be honest there was only one choice. But 2 choices in color of the fabric. But it was fine for our backyard. Got a love seat 2 club chairs a coffee table and 2 chaise lounges plus some kind of table. Can't wait for summer. Now I hope the quality is good for the money we spent. Next time it's bar stools if these pieces are of good quality.,0,78
1998,-0.256362,0.534860,1,0,fp,I brought in a flash drive with a 3-page document I wanted printed. I wanted two-sides color 100 copies. The young man behind the counter whom I'll call Skippy said that it would take over an hour. I get it that they may have been busy so I gave them my phone number and left to do other things. 2 1/2 hours later I still hadn't received a call so I head back. I notice that Skippy is gone but a teenager and an older woman are working the printing counter. They're helping other customers both o...,0,223
1999,0.378196,-0.824340,0,0,tn,Super helpful. Taught me exactly how to gel stain refinish my curio cabinet. So excited to put it back into my renovated house. This is our go to hardware store for knobs hinges and specialty items. Fairly priced. Worth the trip.,0,41


##### Accuracy

In [85]:
accuracy(preds)

0.686

##### Classification report

In [86]:
with open(os.path.join(results_fp, "classification_report.txt")) as report_file:
    print(report_file.read())

              precision    recall  f1-score   support

           0       0.72      0.61      0.66      1000
           1       0.66      0.76      0.71      1000

    accuracy                           0.69      2000
   macro avg       0.69      0.69      0.68      2000
weighted avg       0.69      0.69      0.68      2000




##### Confusion matrix - raw & normalized

In [87]:
confusion_matrix(preds, False)

,Pred = False,Pred = True
,,
Actual = False,615,385
Actual = True,243,757


In [88]:
confusion_matrix(preds, True)

,Pred = False,Pred = True
,,
Actual = False,0.3075,0.1925
Actual = True,0.1215,0.3785


##### Word count comparison

In [89]:
preds.groupby(["category"])["num_words"].mean()

category
fn     80.646091
fp    165.610390
tn     60.707317
tp    221.466314
Name: num_words, dtype: float64

##### Most confident - true positives

In [90]:
most_confident_predictions(preds, "tp")

,category,text,num_words
0,tp,Been livin' in LV since the 90's. finally trekked on down to the Neon Museum only years after it opened to the public! The Neon Museum only has a select number of tickets sold per day so you have to check their website on the day you want to go. It's possible tickets for that day is already sold out! They don't really take walk-ins since each tour guide only takes a certain amount of people with them per session. It's often busy helllooooo - tourist attraction! The day we went there were a n...,514
1,tp,I have no clue what this place is. I went a few days ago for the Femocracy show and I'm not really sure what that was either. I'm not really into the Phoenix art scene so I don't get a lot of the stuff that goes on here so I'll do my best to describe my experience. The Alwun house is just a big house in downtown Phoenix that hosts different art events. When I went the house was filled with erotic art which was eclectic and well. erotic. In back there is a large mostly-covered area with a sta...,231
2,tp,Gears has been around for aaaaaaaaaaaaaaaaaaaaaaagesss. From the outside the big yellow with blue letters really hurt my eyes but the pain just pulls me closer where inside it heals me. Gears is a fully operational Bike Snowboard And Spinning Cycle Shop. At the age of 17 Kevin Wallace started GEARS in 1988 as a little bike shop in Clarkson Train Station on Clarkson Road. Ever since then the place has been passed down to the family. Today GEARS is one of the largest bicycle retailers in Canad...,683
3,tp,If you wanted to change from a commonplace nerd into a rough-kid wayward punker and make the transformation happen over night than you need only make one stop in Vegas. Cash 4 Chaos has the goods that any poser would need for immediate and unearned street-cred! Whoo-hoo! Admittingly in Vegas even the old veteran punkrockers still need to shop here too. There isn't any other place where you can buy CREEPERS DOCS or that Rudimentary Peni T-SHIRT you've been always wanting. It is also a steadfa...,273
4,tp,What's the first thing a girl does when she gets engaged? Well try on wedding dresses of course! I found Almond Tree Wedding Boutique online while searching for bridal stores and immediately made an appointment. I was surprised that not only did I get a email confirmation for my appointment I also received an text from Pam confirming my appointment so I wouldn't forget (how could I with the excitement of being engaged? ) On the date of my appointment my mom and I arrived 15 minutes early (yo...,265
5,tp,I was attending a conference at the San Marcos Hotel in downtown Chandler. Whenever I attend a conference I like to get out and explore the local businesses even if the conference is in a suburb of my very own city. So when the sessions ended @ 5 p.m.on Thursday I went strolling through Chandler's very charming downtown. Because women are biologically programmed to be unable to walk past a shoe store I stopped into Shoe Thrill because happily they stay open until 8 p.m. I was like a kid in a...,269
6,tp,Delicately positioned in the delightful Bloordale Village is a petite vintage clothing shop called Vintage Depot. It's on the the south side of Bloor just east of Lansdowne. When you enter it's racks galore a sea of clothing divided by gender and sorted into categories like windbreakers hoodies athletic army jackets tees shoes etc. The men's tops are sorted by colour which made it pretty easy to find a purple T-shirt. Something I needed desperately. Don't ask. There is a healthy mix of reall...,201
7,tp,This review is based solely on the parking lot out front which is filled with creepy decrepit mannequins arranged in a bizarre tableau involving a swingset a kiddie pool and a mechanical bull. Back in the day they used to pay a Russian chick $7 an hour to swing on the swingset and wave at passers-by. ostensibly to draw in traffic. But now. thanks to the economy the Russian chick's job was outsourced to t

##### Most confident - true negatives

In [91]:
most_confident_predictions(preds, "tn")

,category,text,num_words
0,tn,For all your 'home' needs hello it's Home Depot they have everything! Plus their is always a staff member around to answer all your questions.,25
1,tn,Love this Nordstrom Rack. The store is huge. Well stocked and very clean/organized. Customer service was friendly in checking out. A+.,21
2,tn,Great variety of products. The staff is extremely helpful and friendly and the location is prime. They have a fresh fudge too.,22
3,tn,This is a great location! Great variety and always have a sale section :) Working are friendly and knowlegable.,19
4,tn,Other than the fact that this Marshalls is a little on the smaller side the store is very nice and in typical fashion aplomb with great discounts . Love the clearance section and the fact that it is lever super crowded- like some of the other stores in the area.,50
5,tn,Not bad. Not the best. Usually people are friendly and service is timely. I wasn't impressed with the Drs. But the regular staff is fine. Selection of frames is small.,30
6,tn,Always fast and courteous.,4
7,tn,Good location. Small store but friendly staff and willing to order if necessary. Not overly busy easy parking in lot out front.,22
8,tn,Clean typical store. Hard to find someone knowledgeable to ask about office supplies/equipment sometimes. Located in the same mall as Chipotle Stein Mart Honeybaked Hams etc.,26
9,tn,It isn't there anymore.,4


##### Most confident - false positives

In [92]:
most_confident_predictions(preds, "fp")

,category,text,num_words
0,fp,Nick M. and his wife is the owner of the shop. Find the lumber jack and turn right i to the complex if you are taking Craig. if you are going up then make a left on revere and make right on the complex. go up a building unit and look for a yellow truck. next to a gun store. Bike worx is across the skate park. Nick has pricings that matches or almost matches danscomp. com. His complete bikes beit a mountain bike or bmx is reasonable. His park bikes are very new and he does have the latest par...,178
1,fp,This is the third time that Andrew and his growing company have cleaned our windows and deck. They not only do a great job but they have leverage technology to provide spotless windows. Andrew started out washing windows as a college student. A very personable guy he innovated and grew this part-time effort to earn money into a full-time business. In just three years he grew from Andrew's Window Cleaning into Wash-It-Pros. They do windows but now do whole house exterior and deck cleaning usi...,283
2,fp,I wasn't blown away by this GAP. Let's face it GAP's popularity fashion sense and quality have all gone downhill over the years. It's almost as if Old Navy has switched to becoming the line-ups flagship brand. (Keeping in mind it GAP and Banana Republic are all in the same brand supposedly increasing in quality and price in that order. ) Old Navy is still cheap but the quality is better these days. GAP is still the same price but the variety and quality have declined. At least at this locati...,263
3,fp,Wait wait I thought I had already reviewed Paper Skyscraper? Well it's certainly been a long time coming. All I can say is love it! They've got all the knick-knacks imaginable in this world. Excellent joke gifts like bottle openers mounted on wood with a painting of the crotch of a naked man or woman. Plenty of those items you never knew you needed like a bottle opener that looks like a skeleton key or spoons that nest together with magnets and stand upright on your countertop. Who'd a thunk...,245
4,fp,My friends and I had an hour to kill before the Elite Great Gatsby Yelp Event. So they started whining that they wanted food. Then we drove by this sex shop and they wanted to go there and check it out. No food. No sex shop. No food. No sex shop. I finally said enough we are going to the sex shop and then getting something to eat and if they didn't like that I was turning the car around and we were going home. That settled we pulled into the parking lot. This place has definitely been here f...,360
5,fp,We went to see the Friday Night Gunfights. I learned that this is only place in the world utilizing Simunition Sport a non-lethal training ammunition gunfight experience using real guns. When you first walk in the staff take you to the hookah lounge until they're ready. We had wine and cocktails and the price was decent around $6 -$8. They have a small menu consisting of veggie samosas kababs etc. We didn't have time to try their Hookah. Around 8:10pm the owner gave introduction and they tak...,172
6,fp,I have visited this Old Navy location four or five times over the past year and have never had an issue. In fact a friend and I visited the store around Christmastime and the staff were extremely friendly and helpful. However this was the not my experience yesterday. I drug my husband along so I could look for new jeans and maybe find some sweaters / winter clothes on sale. There were signs up exclusively for Old Navy credit card users which I am not. However the 'sale' of buy two get one fr...,376
7,fp,(I have only been to one exhibit here so until I see how other exhibits fare 3 stars. Also this is a piece I wrote for a magazine that never published it. ) 'People either love Andy Warhol or they hate him' quipped Warhol Out West exhibit tour guide Gina 'And I think he loved that people felt that way about him. ' It's easy to see why people feel so strongly. In today's world of Instagram filters and Pho

##### Most confident - false negatives

In [93]:
most_confident_predictions(preds, "fn")

,category,text,num_words
0,fn,Roses have always been fresh and beautiful! Arrangements are pretty and delivery is always on time.,16
1,fn,Now at Town and Country a whole new Rack to entice me into spending money. Stopped by for the grand opening today and I can only hope they keep the place as nicely organized as it currently is. Same quality as you'd expect from Nordstrom great selection and generally 1/3 to 1/2 the price. No complaints. None at all.,59
2,fn,Huge selection friendly employees great holiday items lots of coupons and special deals Walgreens is the go-to store for middle of the night cold medications and the random gallon of milk and some bread. They are consistently a 4 star store.,41
3,fn,Called at 8 am for a estimate was given estimate over the phone and scheduled the spring repair the same day after work. Showed up on time and work was completed in a timely. Would recommend.,36
4,fn,Love coming here drunk or sober. Every year for my birthday I force someone to play Buck Hunt with me. Fun place to kill some time and quarters.,28
5,fn,Winners is terrific if you don't know the glory that is Loehmann's. On my last visit I found nothing on the racks worthy of entering the dressing room to try on.,31
6,fn,Got the camera back from being repaired within a week. He was able to fix it for the price originally quoted and the camera works beautifully! Will definitely return for any future repairs. Morris really knows his stuff and doesn't mind sharing his knowledge so you know what you can expect from the repair.,54
7,fn,good mall for walking. there is a costco movie theater and decent food court.,14
8,fn,Sadly this location is now closed. I used to buy all my cables and earphones here. super cheap. I think the closest one is near yorkdale mall.,27
9,fn,Have stayed here a couple of times over the years. At the time I thought Bally's was quite nice. Nowadays I do not. For the price of the rooms I would not recommend staying here. There are much nicer places on the Las Vegas Strip to spend the night at for the same price or lower. What I do like about Bally's is the location. Their location on the Strip is walking distance to all the major hotels and casinos. Will still drop by here if we are taking a stroll of the Las Vegas Strip.,96


### Amazon large model, 2 layers finetuned on Bilal

In [94]:
pretraining_model = "amazon_finetune_LARGE"
finetuning_dataset = "bilal"
n_layers = 2

model_fp = os.path.join(MODEL_DIR, f"{pretraining_model}_{n_layers}_LAYERS_FINETUNED_ON_{finetuning_dataset}", "results")
timestamps = [f for f in os.listdir(model_fp) if f.startswith("2022")]
assert len(timestamps) == 1
results_fp = os.path.join(model_fp, timestamps[0])
preds_path = os.path.join(results_fp, "predictions.csv")
preds = pd.read_csv(preds_path, encoding="utf-8")
test_set = datasets[finetuning_dataset]["test"][["sentence", "label"]]
preds["text"] = test_set["sentence"]
preds["label"] = test_set["label"]
preds["num_words"] = [len(t.split()) for t in preds["text"]]
preds.tail()

,pred_prob_0,pred_prob_1,yhat,y,category,text,label,num_words
1995,0.063274,-0.305606,0,0,tn,Big sale this week. All sort of little gadets that you really don't need but you want them anyways. Prices are really cheap and new stuff brought out all day long. Items for your yard all kinds of tools and hardware holiday decorations and polo shirts and t- shirts. Lots of fun hunting thru boxes and time really fly's fast.,0,60
1996,0.232174,-0.709767,0,0,tn,The new owner and management are great. I didn't tell them I knew anything about cars when I brought mine in to get fixed cause I wanted to see if they would be strait forward and honest with me or try to sell me a bunch of BS I don't need. They were 100% honest about the problem and the price was really good. I'm happy I brought my car here.,0,71
1997,-0.141196,0.240564,1,0,fp,Came here to check out their Patio Furniture. To be honest there was only one choice. But 2 choices in color of the fabric. But it was fine for our backyard. Got a love seat 2 club chairs a coffee table and 2 chaise lounges plus some kind of table. Can't wait for summer. Now I hope the quality is good for the money we spent. Next time it's bar stools if these pieces are of good quality.,0,78
1998,-0.301840,0.498640,1,0,fp,I brought in a flash drive with a 3-page document I wanted printed. I wanted two-sides color 100 copies. The young man behind the counter whom I'll call Skippy said that it would take over an hour. I get it that they may have been busy so I gave them my phone number and left to do other things. 2 1/2 hours later I still hadn't received a call so I head back. I notice that Skippy is gone but a teenager and an older woman are working the printing counter. They're helping other customers both o...,0,223
1999,0.078677,0.245192,1,0,fp,Super helpful. Taught me exactly how to gel stain refinish my curio cabinet. So excited to put it back into my renovated house. This is our go to hardware store for knobs hinges and specialty items. Fairly priced. Worth the trip.,0,41


##### Accuracy

In [95]:
accuracy(preds)

0.645

##### Classification report

In [96]:
with open(os.path.join(results_fp, "classification_report.txt")) as report_file:
    print(report_file.read())

              precision    recall  f1-score   support

           0       0.69      0.53      0.60      1000
           1       0.62      0.76      0.68      1000

    accuracy                           0.65      2000
   macro avg       0.65      0.65      0.64      2000
weighted avg       0.65      0.65      0.64      2000




##### Confusion matrix - raw & normalized

In [97]:
confusion_matrix(preds, False)

,Pred = False,Pred = True
,,
Actual = False,529,471
Actual = True,239,761


In [98]:
confusion_matrix(preds, True)

,Pred = False,Pred = True
,,
Actual = False,0.2645,0.2355
Actual = True,0.1195,0.3805


##### Word count comparison

In [99]:
preds.groupby(["category"])["num_words"].mean()

category
fn    110.556485
fp    142.895966
tn     63.877127
tp    211.332457
Name: num_words, dtype: float64

##### Most confident - true positives

In [100]:
most_confident_predictions(preds, "tp")

,category,text,num_words
0,tp,Updating my review of Ring Music -2 stars. a few things here lately have been less than stellar for me. 1. I bought a bunch of strat parts here. They were really expensive (more than 12th fret) and he sold me the wrong pots. I knew I needed 1meg pots and he sold me 500k pots and said they would be fine then they only offered me half back for them (I just kept them I'll put them in something else). They also charge 50c a screw for pickguard screws 23$ for an Ernie Ball 5-way switch. not even ...,415
1,tp,Pretty choked last night with all the 'sales' staff I encountered. Apparently Macy's ( or the bitchy useless employees I had the misfortune of having to deal with ) don't want sales. I tried 4 times to pay and nobody wanted to take my money. It was 1045 and they announced the store was closing and to take final selections to sales associates to pay. I had some Free People clothing I wanted to buy to the girl at the cash desk directly across from Free People ( the girl had straight orange hai...,635
2,tp,I'm only giving this a star because the install guys are decent the rest sucks. Normally I'd give this a zero but i can't so. I've ordered 2 recent PPV's (one UFC one WWE) and when you order it it says 'purchase and record this show' which is good because I work during the time it airs live. Both shows would unexpectedly pause during the main event and not go any further. The first time we called for the UFC situation they rebooted the box remotely and I missed the fight because of the amoun...,314
3,tp,RC Willey is a Berkshire Hathaway company and if you're familiar with Berkshire's other furniture companies such as Nebraska Furniture Mart you pretty much know what you're going to find when you walk in the door. They have a nice looking selection of furniture and electronics and at semi-reasonable prices. Electronics and appliance wise I've looked at their inventory but I've ended up buying elsewhere as I was able to beat them on price and features for both a TV and a washer / dryer. Furni...,298
4,tp,The wonderful folks at Graphite helped me say goodbye to an old friend: my 17' 2008 Macbook Pro. This workhorse computer saw me through a lot of hard times from dissertation-writing to starting my own business website and beyond. I had already been to the Apple Store for emergency Rx after getting the black screen of death only to realize that those poor technicians are working within a very tight timeframe and under high pressure. Though knowledgeable and friendly (at least in my particular...,463
5,tp,Dear Lowes: Thank you for selling my fiancee & I a really nice washer and dryer an extended warranty that at $175 for both units is 75% cheaper than what Sears would charge for their warranties and for hooking us up with the stainless steel hoses dryer hose and the gas line kit we needed. Oh and after dropping all that coin we got promised 'Next Day Delivery! '. It's on huge banners in your store it's so easy to read. It's not so easy to do! Well our washer and dryer was bought on Friday and...,458
6,tp,If you're getting OLD-er & need reading glasses because your ophthalmologist broke your heart by telling you your prescription became stronger (heehee ok my heart has healed); then dollar stores are great! They had hundreds of glasses to choose from for only dollar $1 each! I picked up nine (9) different styles/colors. They do have more masculine styles available too. So much cheaper than other non-dollar stores you can buy at. You can definitely have backups in different rooms at home work ...,144
7,tp,Best Buy is one of those places where you have to know exactly what you're looking for. Otherwise some primordial ball-scratcher will attempt to upsell you on something else by reading the box he's holding before you get a chance to! You know in order to appear knowledgeable without actually being so. I called the other day looking for a specific item and after the guy kept fumbling the name on the phone

##### Most confident - true negatives

In [101]:
most_confident_predictions(preds, "tn")

,category,text,num_words
0,tn,Excellent eclectic selection of old postcards and frame-able prints in addition to a wide array of book titles.,18
1,tn,What a great place. I need to buy one of the CLE winter hats one of these days! I hope this place does well.,24
2,tn,It isn't there anymore.,4
3,tn,Checkout lines were so long but they had the rug we came to buy. They need more than three checkout lines open at night.,24
4,tn,An average gas station. Good and clean inside. Nothing really special to distinguish it from most other gas stations.,19
5,tn,Very unprofessional when dealing with patients really people in general who are in need of the help from the pharmacy. Now I have been disappointed multiple times- reason why I have to mention it on the internet.,37
6,tn,Love the selection prices and supplies are amazing. This is great place for unique tiles and also for common ones that are priced reasonable.,24
7,tn,I've taken a number of vacuums here and they always do it right. They are gas with service and they are well priced.,23
8,tn,I lov w coming to this dispensary its always clean and the customwr service is a plus sign. You relly get the things tou need for a low price.,29
9,tn,I go to this office depot all the time. Nice clean and very helpful. Also right around the corner from my house.,22


##### Most confident - false positives

In [102]:
most_confident_predictions(preds, "fp")

,category,text,num_words
0,fp,I really hate going to Costco and I reaaaaaaaly hate when I have to go for gas. It starts when you pull in the parking lot and all common sense goes out the window. The lines are ridiculously long. Then there is always that jerkoff that can't wait 2 extra seconds for one car to pull away so they can pull forward and the next 2 cars can go. I really want to beat that person up. But I am not a jerk off so I sit in my car and stew about what an idiot they are. If you are that jerkoff please thi...,170
1,fp,A quiter branch of the chain geared more towards the sedate suburban lifestyle. This Running Room is located along a strip of stores across the street from Promenade Mall overlooking a Wal-Mart with built-in McDonalds. It is surrounded by an All-You-Can Eat Japanese restaurant a Marble Slab Creamery two falafel joints and the worst bagel restaurant in the world. And you wonder why suburbanites tend to be heavier than their urban counterparts? I'll spare you jokes about the need for stretch p...,160
2,fp,This is the 2nd Hobby Lobby to open in Vegas. It's slightly larger than the Henderson store a bit cleaner but the friendly staff you find in the Henderson location is also found here. It's also a bit easier IMHO to get in and out of as it's closer to the 215 than the Henderson location. Don't forget to go online and print off your 40% off coupon before you go.,71
3,fp,About three years ago Judy aand I met Nancy Doney at the Nordstrom's in the Hosiery Department in Chandler Fashion Center. We like shopping there for a variety of Hue jeggings. Nancy was always warm welcoming to us genuinely happy to see us and above all extremely happy to assist us in any way she could. Now Nancy works at Nordstrom Rack on Chandler Boulevard where she continues to exceed by helping customers any way that she can. Nancy's customer service excels that of anyone I've ever met ...,273
4,fp,The salesgirl told me they're based out of Florida and is family owned. I tried on some body butter on my skin and it felt good. I bought 2 thin sliced bars of soap which cost over $5 to over $6/bar - kind of costly and 3 small bath bombs. She told me their bombs are jet safe for the jacuzzi and do not foam - meant to be moisturizing. All the above for $24.73.Similar to a LUSH store. They have shaving creams bar shampoos as well. The lemongrass bombs smelled really good she says one small bo...,103
5,fp,I know I'm in the minority but I'm not a fan of H&M. Maybe I'm not its target demographics but i'll be blunt and say the quality sucks. I understand 3 of my lattes probably cost the same as a shirt but at least my drink is well made. This particular store is pretty big but I feel the layout is purposely done so that you're zig zagging around the store. It's not fluid or as open as a Club Monaco or Banana Republic. The shelves are filled with clothes and sometimes they're rammed in and it isn...,164
6,fp,This place artfully combines my love of wine with my appreciation for efficient categorization. The prices for their wine are even lower than Walmart (and for those who have AMX cards that give a higher percentage back for 'groceries' the wise people at AMX have deemed this a grocery store!). Unless you are truly confident that you will only be buying one bottle of wine I would highly recommend grabbing a cart at the entrance. I know I have to walk through several tempting displays to get to...,300
7,fp,I can take this place or leave it. Fry's across the street just remodeled so I may start going there- and I used to hate doing that! The way to shop here is to look for yellow price tags which are 'manager's Specials' and they are the only way you aren't going to get price gouged. If you were to do a 'cart-full' grocery shop here you'd be in trouble even with the discount card. it's just not a cheap place to shop at all. They DO have a butcher (who is never around when I shop at night) but t...,304
8,fp,Lock N Load is my favorite gun store

##### Most confident - false negatives

In [103]:
most_confident_predictions(preds, "fn")

,category,text,num_words
0,fn,Love coming here drunk or sober. Every year for my birthday I force someone to play Buck Hunt with me. Fun place to kill some time and quarters.,28
1,fn,Roses have always been fresh and beautiful! Arrangements are pretty and delivery is always on time.,16
2,fn,I couldn't schedule with my fav stylist at Suite One.,10
3,fn,I was in the area and decided too go to this Giant Eagle this time. It is one of the bigger supermarkets int he area. They had most of what I needed and was in and out pretty quickly. The prices are ok and the customer service is fast and friendly. If there is crowded there could be a wait at the registers. I would go back to this Giant Eagle again.,72
4,fn,My friend and I stopped in to get a consultation on some tattoos we were thinking about getting and I was very impressed with the customer service and knowledge of the staff. We went through the artist portfolios and felt really confident about getting a tattoo here. I haven't gotten it done yet but I plan on updating my review when I do.,63
5,fn,Super nice employees fast repairs and very reasonable prices compared to anywhere else for galaxy s3 glass replacement. Highly recommended.,20
6,fn,A full hour to fill a prescription. Seriously? I'll go somewhere else next time.,14
7,fn,This is my new go-to location to get my eyebrows threaded! Love this location and the amazing customer service The ladies here are so nice and accommodating. If you are in this area I would definitely recommend this shop.,39
8,fn,Usually I like this store but not this time. the whole time there was a dude standing outside my dressing room and the store employees could care less so of course it was a bit uncomfortable :/.,37
9,fn,Sadly this location is now closed. I used to buy all my cables and earphones here. super cheap. I think the closest one is near yorkdale mall.,27


### Amazon large model, 4 layers finetuned on Bilal

In [104]:
pretraining_model = "amazon_finetune_LARGE"
finetuning_dataset = "bilal"
n_layers = 4

model_fp = os.path.join(MODEL_DIR, f"{pretraining_model}_{n_layers}_LAYERS_FINETUNED_ON_{finetuning_dataset}", "results")
timestamps = [f for f in os.listdir(model_fp) if f.startswith("2022")]
assert len(timestamps) == 1
results_fp = os.path.join(model_fp, timestamps[0])
preds_path = os.path.join(results_fp, "predictions.csv")
preds = pd.read_csv(preds_path, encoding="utf-8")
test_set = datasets[finetuning_dataset]["test"][["sentence", "label"]]
preds["text"] = test_set["sentence"]
preds["label"] = test_set["label"]
preds["num_words"] = [len(t.split()) for t in preds["text"]]
preds.tail()

,pred_prob_0,pred_prob_1,yhat,y,category,text,label,num_words
1995,0.218308,-0.401459,0,0,tn,Big sale this week. All sort of little gadets that you really don't need but you want them anyways. Prices are really cheap and new stuff brought out all day long. Items for your yard all kinds of tools and hardware holiday decorations and polo shirts and t- shirts. Lots of fun hunting thru boxes and time really fly's fast.,0,60
1996,0.115872,-0.473677,0,0,tn,The new owner and management are great. I didn't tell them I knew anything about cars when I brought mine in to get fixed cause I wanted to see if they would be strait forward and honest with me or try to sell me a bunch of BS I don't need. They were 100% honest about the problem and the price was really good. I'm happy I brought my car here.,0,71
1997,-0.214237,0.291467,1,0,fp,Came here to check out their Patio Furniture. To be honest there was only one choice. But 2 choices in color of the fabric. But it was fine for our backyard. Got a love seat 2 club chairs a coffee table and 2 chaise lounges plus some kind of table. Can't wait for summer. Now I hope the quality is good for the money we spent. Next time it's bar stools if these pieces are of good quality.,0,78
1998,-0.484472,0.679018,1,0,fp,I brought in a flash drive with a 3-page document I wanted printed. I wanted two-sides color 100 copies. The young man behind the counter whom I'll call Skippy said that it would take over an hour. I get it that they may have been busy so I gave them my phone number and left to do other things. 2 1/2 hours later I still hadn't received a call so I head back. I notice that Skippy is gone but a teenager and an older woman are working the printing counter. They're helping other customers both o...,0,223
1999,0.191227,0.065142,0,0,tn,Super helpful. Taught me exactly how to gel stain refinish my curio cabinet. So excited to put it back into my renovated house. This is our go to hardware store for knobs hinges and specialty items. Fairly priced. Worth the trip.,0,41


##### Accuracy

In [105]:
accuracy(preds)

0.683

##### Classification report

In [106]:
with open(os.path.join(results_fp, "classification_report.txt")) as report_file:
    print(report_file.read())

              precision    recall  f1-score   support

           0       0.73      0.59      0.65      1000
           1       0.65      0.78      0.71      1000

    accuracy                           0.68      2000
   macro avg       0.69      0.68      0.68      2000
weighted avg       0.69      0.68      0.68      2000




##### Confusion matrix - raw & normalized

In [107]:
confusion_matrix(preds, False)

,Pred = False,Pred = True
,,
Actual = False,588,412
Actual = True,222,778


In [108]:
confusion_matrix(preds, True)

,Pred = False,Pred = True
,,
Actual = False,0.294,0.206
Actual = True,0.111,0.389


##### Word count comparison

In [109]:
preds.groupby(["category"])["num_words"].mean()

category
fn     82.175676
fp    159.468447
tn     60.193878
tp    217.228792
Name: num_words, dtype: float64

##### Most confident - true positives

In [110]:
most_confident_predictions(preds, "tp")

,category,text,num_words
0,tp,I'm so happy that this is the newest Giant Eagle to be updated and get the Market District treatment! The grand opening was January 22 2015. Now that the word is out about the place being finished it is always packed and finding a parking space can be a chore. I just don't understand why it took Giant Eagle so long to turn this location into a Market District. They are headquartered in the Fox Chapel/ O'hara Township area after all. It's set up pretty much just like the somewhat larger Robin...,404
1,tp,Friends don't let friends drink and paint. Joking. Here at Buzz'N Brushes drinking and painting is encouraged. We came out here to attend Kayla M's UYE which was a blast. She's been here before as well as Roger T. Both of them gave positive reviews and since I trust their judgment I knew a night out at Buzz'N would be a fun one. What to Expect? -Inside the Orleans specifically located across from their buffet -$30 includes painting lessons 2 drinks fun/partaying and a masterpiece that you ge...,400
2,tp,Updating my review of Ring Music -2 stars. a few things here lately have been less than stellar for me. 1. I bought a bunch of strat parts here. They were really expensive (more than 12th fret) and he sold me the wrong pots. I knew I needed 1meg pots and he sold me 500k pots and said they would be fine then they only offered me half back for them (I just kept them I'll put them in something else). They also charge 50c a screw for pickguard screws 23$ for an Ernie Ball 5-way switch. not even ...,415
3,tp,If you wanted to change from a commonplace nerd into a rough-kid wayward punker and make the transformation happen over night than you need only make one stop in Vegas. Cash 4 Chaos has the goods that any poser would need for immediate and unearned street-cred! Whoo-hoo! Admittingly in Vegas even the old veteran punkrockers still need to shop here too. There isn't any other place where you can buy CREEPERS DOCS or that Rudimentary Peni T-SHIRT you've been always wanting. It is also a steadfa...,273
4,tp,The Noob's guide to a $20 massage at Bali! If you guys reading this are like me you would consider yourself very INEXPERIENCED when it comes to massages in general. And of course you've heard plenty of stories about the shady Asian Massage parlors in Vegas LOL. Since I am now an expert (jk only been one time lol) I will HELP you out and let you know what to expect! 1) I showed up with no appointment but was promptly welcomed and walked to the back. Maybe during prime time you might have to w...,733
5,tp,Guess I'm the one to break the 5 star streak. Sorry guys. Lets just get to the review and you guys be the judge. Remember this is IMO. I support all B&M shops so this is just my constructive criticism. So the shorty has been in between mods and rda for the past 2 weeks and today we decided to shoot to this B&M to check out the mods they had so we compare sizes weight ect. It's by the house in fact only about 2 miles or so away. Did not know they were there. Hidden between the coldstone and z...,553
6,tp,Generally I'm not a huge fan of Macy's just because I'm enamored of the brand itself which is why I feel like so many people love Macy's. Yes you have to give them props for longevity the parade in NYC Miracle on 34th St. etc. but companies simply cannot rest on their laurels and assume that bad service is ever OK. Which leads me to why I'm reviewing this location and giving 2 stars. When me mums was out for Thanksgiving a few weeks ago we went there because she wanted to look at some things...,465
7,tp,I was attending a conference at the San Marcos Hotel in downtown Chandler. Whenever I attend a conference I like to get out and explore the local businesses even if the conference is in a suburb of my very own city. So when the sessions ended @ 5 p.m.on Thursday I went strolling through Chandler's very charming downtown. Because women are biologically programmed to be unable to walk past a shoe store I s

##### Most confident - true negatives

In [111]:
most_confident_predictions(preds, "tn")

,category,text,num_words
0,tn,Excellent eclectic selection of old postcards and frame-able prints in addition to a wide array of book titles.,18
1,tn,An average gas station. Good and clean inside. Nothing really special to distinguish it from most other gas stations.,19
2,tn,I have been going here for years. they have never done my nail terrible. always respectful of my time. and do an awesome services. I have my favorite technicians but don't we all.,33
3,tn,What can you say it is Home Depot. Just awesome for any home improvement projects. You cannot beat the price availability and selection at the store.,26
4,tn,Super friendly staff Nice selection of products. Unlike other locations the people here seem to be less pushy to get their goods.,22
5,tn,Cheap. Hit or miss but cmon. It's that kind of store . Always have the dog bagels that my pup has to have. People are friendly.,26
6,tn,Pretty standard as far as a T. J. 's goes! Produce is always fresh with great options. The only hit on it is the parking garage. when it's busy it is a hassle. I don't live nearby otherwise I would frequent here much more often.,45
7,tn,This is a nice Walgreens easy access since the parking lot connects to the other stores all the way from Van Buren to I-10.,24
8,tn,Lowes is one of my favorite stores. The Surprise location is nice and usually not too crowded. I like that Lowes will price match too.,25
9,tn,Good location. Small store but friendly staff and willing to order if necessary. Not overly busy easy parking in lot out front.,22


##### Most confident - false positives

In [112]:
most_confident_predictions(preds, "fp")

,category,text,num_words
0,fp,This is the kind of experience that you could rate anywhere from 1 star to 5 stars. Why? Because its mad crazy! It's a freak show. it's tacky. It's got live bands. it's got talented artists impersonators show girls Elvii of all kinds tourists galore a laser light show that ranges from weird to awesome overpriced objects for sale lots and lots of casinos and eateries. I mean you have to give it 5 stars because it does weird so well. My favorites: 1) the saxophone player. He definitely loves h...,562
1,fp,About three years ago Judy aand I met Nancy Doney at the Nordstrom's in the Hosiery Department in Chandler Fashion Center. We like shopping there for a variety of Hue jeggings. Nancy was always warm welcoming to us genuinely happy to see us and above all extremely happy to assist us in any way she could. Now Nancy works at Nordstrom Rack on Chandler Boulevard where she continues to exceed by helping customers any way that she can. Nancy's customer service excels that of anyone I've ever met ...,273
2,fp,This place artfully combines my love of wine with my appreciation for efficient categorization. The prices for their wine are even lower than Walmart (and for those who have AMX cards that give a higher percentage back for 'groceries' the wise people at AMX have deemed this a grocery store!). Unless you are truly confident that you will only be buying one bottle of wine I would highly recommend grabbing a cart at the entrance. I know I have to walk through several tempting displays to get to...,300
3,fp,(I have only been to one exhibit here so until I see how other exhibits fare 3 stars. Also this is a piece I wrote for a magazine that never published it. ) 'People either love Andy Warhol or they hate him' quipped Warhol Out West exhibit tour guide Gina 'And I think he loved that people felt that way about him. ' It's easy to see why people feel so strongly. In today's world of Instagram filters and Photoshop people may look at his work and say 'I can do that! ' or 'that's not art! ' But do...,439
4,fp,So I am updating my original review from 2 stars to 4. Several Home Depot employees have reached out to me to rectify the issues my husband I had with this purchase. Even the new store manager who wasn't even at this store when all of this took place took time out of his day to call. My negative review wasn't about getting something for nothing either. It was about an associate who didn't handle a situation properly. All of the calls and emails I have received have proved to me that as a who...,458
5,fp,The latest Livestock location -- not sure whether to call it 'Livestock Roncesvalle' or 'Livestock Round 6' as posted on the store front. I guess Round 6 sorta makes sense -- 3 locations in Vancouver one on Spadina they are counting one as their online store -- I guess they rather forget the failed Alberta expansion. And now it's en vogue to called Toronto the '6ix'. and if you really want to get into the numbers. the street address is '604' reversed. Anyways it's the similar product and sta...,451
6,fp,Upon leaving for work the sprocket on my garage door opening system broke. After doing some research Performance Door Service seemed to be the most recommended business for my needs in Peoria. I called and Alesha was able to help me and schedule someone to come out within the hour. 15 minutes before getting to my house the technician Rick called to let me know he was on the way. He was very friendly upon arrival and looked at our problem and gave me all my options. Basically repairs (we had ...,317
7,fp,I can take this place or leave it. Fry's across the street just remodeled so I may start going there- and I used to hate doing that! The way to shop here is to look for yellow price tags which are 'manager's Specials' and they are the only way you aren't going to get price gouged. If you were to do a 'cart-full' grocery shop here you'd be in trouble even with the discount card. it's just not a cheap plac

##### Most confident - false negatives

In [113]:
most_confident_predictions(preds, "fn")

,category,text,num_words
0,fn,I couldn't schedule with my fav stylist at Suite One.,10
1,fn,Winners is terrific if you don't know the glory that is Loehmann's. On my last visit I found nothing on the racks worthy of entering the dressing room to try on.,31
2,fn,Love coming here drunk or sober. Every year for my birthday I force someone to play Buck Hunt with me. Fun place to kill some time and quarters.,28
3,fn,Roses have always been fresh and beautiful! Arrangements are pretty and delivery is always on time.,16
4,fn,Lots of action here! Lots of restaurants and lots of shops! Top name! This is the place to go to! Bring lots of money or a boyfriend with lots of money not one who says he is going to take you shopping there and not show up.,47
5,fn,On the occasion that I needed some help the staff went out of their way to find what I needed. I am impressed with the diversity of merchandise at this location as well.,33
6,fn,I've never bought a single ting at a Club Monaco. I go in from time to time to see but I just can't get over their prices and the up-tightness of the staff.,33
7,fn,I like this store because it's close to my house. The staffs there are nice and helpful. I have done my homework online knowing what I want however they don't have much in the stores. End up I left empty handed and ordered online.,44
8,fn,good mall for walking. there is a costco movie theater and decent food court.,14
9,fn,They have products most stores don't have. Sometimes you can get the same products on E-Bay for the same price. Never listen to advice from anyone in this store though. Ever.,31


### Base BERT model, 2 layers finetuned on Yelp

In [ ]:
pretraining_model = "bert_base"
finetuning_dataset = "yelp"
n_layers = 2

model_fp = os.path.join(MODEL_DIR, f"{pretraining_model}_{n_layers}_LAYERS_FINETUNED_ON_{finetuning_dataset}", "results")
timestamps = [f for f in os.listdir(model_fp) if f.startswith("2022")]
assert len(timestamps) == 1
results_fp = os.path.join(model_fp, timestamps[0])
preds_path = os.path.join(results_fp, "predictions.csv")
preds = pd.read_csv(preds_path, encoding="utf-8")
test_set = datasets[finetuning_dataset]["test"][["text", "label"]]
preds["text"] = test_set["text"]
preds["label"] = test_set["label"]
preds["num_words"] = [len(t.split()) for t in preds["text"]]
preds.tail()

##### Accuracy

In [ ]:
accuracy(preds)

##### Classification report

In [ ]:
with open(os.path.join(results_fp, "classification_report.txt")) as report_file:
    print(report_file.read())

##### Confusion matrix - raw & normalized

In [ ]:
confusion_matrix(preds, False)

In [ ]:
confusion_matrix(preds, True)

##### Word count comparison

In [ ]:
preds.groupby(["category"])["num_words"].mean()

##### Most confident - true positives

In [ ]:
most_confident_predictions(preds, "tp")

##### Most confident - true negatives

In [ ]:
most_confident_predictions(preds, "tn")

##### Most confident - false positives

In [ ]:
most_confident_predictions(preds, "fp")

##### Most confident - false negatives

In [ ]:
most_confident_predictions(preds, "fn")

### Base BERT model, 4 layers finetuned on Yelp

In [115]:
pretraining_model = "bert_base"
finetuning_dataset = "yelp"
n_layers = 4

model_fp = os.path.join(MODEL_DIR, f"{pretraining_model}_{n_layers}_LAYERS_FINETUNED_ON_{finetuning_dataset}", "results")
timestamps = [f for f in os.listdir(model_fp) if f.startswith("2022")]
assert len(timestamps) == 1
results_fp = os.path.join(model_fp, timestamps[0])
preds_path = os.path.join(results_fp, "predictions.csv")
preds = pd.read_csv(preds_path, encoding="utf-8")
test_set = datasets[finetuning_dataset]["test"][["text", "label"]]
preds["text"] = test_set["text"]
preds["label"] = test_set["label"]
preds["num_words"] = [len(t.split()) for t in preds["text"]]
preds.tail()

,pred_prob_0,pred_prob_1,yhat,y,category,text,label,num_words
5889,-1.273377,1.274733,1,1,tp,"After months of construction, Yoshi has finally opened. At this point, my anticipation was running so high they could be serving neatly wrapped stones and I'd probably give it a good review. What they do serve, however, is frozen yogurt and sushi. \n\nThe frozen yogurt is the same as in most other asian style frozen yogurt places. They have the basic tart flavor and many others. Yogurt plus toppings is 49 cents/ounce so expect to pay around $4-5 for a decent portion. I thought their topping ...",1,269
5890,0.933077,-0.763021,0,0,tn,This is our favorite restaurant. I always get the lobster ravioli and we usually start with risotto balls. We usually try something different each time we go and loved everything we had from them. The cocktails are great too. They also have the best service around. Everyone is so kind and professional. They're the best.,0,55
5891,0.240561,-0.024416,0,0,tn,"My boyfriend and I went to the drunken crab for our first time tonight. We've never been to a place with this style of eating, so we were super grateful when our waitress came over and explained the menu to us. We didn't want to get super messy so she was very accommodating and gave us all the sauces we wanted on the side for dipping. The king crab was a dream come true, and the calamari was the perfect combo of chewy and crispy. We are huge fans of unique beers and were very happy with the ...",0,106
5892,-0.873770,0.719320,1,1,tp,"So I found out from one of our Uber drivers that most restaurants don't want to serve boiled crawfish because ""they don't want to deal with it"" which is understandable. We came to NOLA during peak crawfish season and were on the hunt. Being from the Bay Area (CA), we are used to the Vietnamese style crawfish so it's literally swimming in either oil or butter. I honestly don't like it because it gives me the worst heartburn and you have to rely on the sauce for flavor. Big EZ Seafood doesn't ...",1,194
5893,0.981642,-0.664830,0,0,tn,I'm from the tri-state area and this pizza is the closest that I've found (in the Franklin area) to the pizza I grew up with! Highly recommend!!,0,27


##### Accuracy

In [116]:
accuracy(preds)

0.7650152697658635

##### Classification report

In [117]:
with open(os.path.join(results_fp, "classification_report.txt")) as report_file:
    print(report_file.read())

              precision    recall  f1-score   support

           0       0.77      0.76      0.77      2992
           1       0.76      0.77      0.76      2902

    accuracy                           0.77      5894
   macro avg       0.77      0.77      0.77      5894
weighted avg       0.77      0.77      0.77      5894




##### Confusion matrix - raw & normalized

In [118]:
confusion_matrix(preds, False)

,Pred = False,Pred = True
,,
Actual = False,2265,727
Actual = True,658,2244


In [119]:
confusion_matrix(preds, True)

,Pred = False,Pred = True
,,
Actual = False,0.384289,0.123346
Actual = True,0.111639,0.380726


##### Word count comparison

In [120]:
preds.groupby(["category"])["num_words"].mean()

category
fn     66.718845
fp    172.393398
tn     49.898013
tp    238.413993
Name: num_words, dtype: float64

##### Most confident - true positives

In [121]:
most_confident_predictions(preds, "tp")

,category,text,num_words
0,tp,"FRANK LLOYD WRIGHT IS NOT AMUSED for function truly follows form at Charcoal. This is one of those trendy over-designed joints with bad value for the money. The de novo building has to be one of the most ill conceived, bad investments in recent memory. Who would construct such a fortress for a low margin business as burgers? Not surprisingly they seem to be scratching-out repayment by outrageously over-pricing their product.\n\nA SLIDER (?) WITH FRIES $11.50!!! But the fries were served ...",170
1,tp,"Wow, I'm hooked! This review started off as an evaluation of three terrific Greek restaurants located within the NW Hillsborough / Westchase area. At great personal expense to my own waistline and bank account, I forced myself to eat at each of the three restaurants and then render an opinion for the benefit of my fellow Yelpsters. The restaurants under consideration were The Hungry Greek, Little Greek and Pappas Market Cafe. I've already written reviews for Pappas and Little Greek, and so ...",562
2,tp,"tl;dr a Mediterranean takeout restaurant with large platters at a slightly premium\n\nIn a neighborhood that's begging for more diverse food options, Falafel Time is a welcome addition. Falafel Time popped up in Graduate Hospital seemingly out of nowhere, replacing a BBQ restaurant that had been there for years.\n\nBefore you go, take note that Falafel Time is first and foremost a takeout restaurant. The tiny space is divided in half into kitchen and dining area, offering a mere 2 metal tabl...",301
3,tp,"No. That wasn't George W. at the round table in the corner plotting his next move in the campaign to evict the British from the Original Thirteen. Although the Inn named for his most daring plan feels like it might have been around to provide succor to our troops, it actually came later. Still, if patriotic memories served with drink and sustenance are your thing, the Washington Crossing Inn will make you a very satisfied customer. The place just feels so authentic and historically ground...",344
4,tp,"For someone who's thrilled to immerse himself in a $5.99 prime rib and fries dinner at Cal-Neva's Top Deck coffee shop, the early-bird dinner at Harrah's Steakhouse ($25.50) comes as a culture shock.\n\nBut, OK, I'll shed my reeking-of-smoke khaki tee shirt and pull over a long-sleeved henley top--but never change out of my blue jeans--and try to put on a dignified front for the tuxedoed servers in the dimly lit, wood-accented and clubby dining room. It's a splurge that our travel partners ...",311
5,tp,"EXQUISITE! \n\nCentral Table Food Hall is a food court for a foodie with discriminating taste. I was fortunate enough to attend a Tasting Dinner for Yelp Elite last week and after being teased with all those delectable tastings, I was anxious to return. I am also proud that Central Table is my 600th Yelp review. \n\nThe ambiance is magnificent and impressive as a Grand Hall should be. It is vast and open with massive amounts of natural light spilling in through the windows. (The natural lig...",696
6,tp,"Better than Woofie's...\n\nI wish this place was downtown St Louis. In fact, open up a franchise in Soulard or South Grand, (specifically where the old Fat Toney's BBQ was near the Farmer's market) and you absolutely will be a Millionaire in short order. Are any of you young entrepeneurs listening? I'm telling you that with the proximity to the Market and with late night hours you will be swimming in greenbacks!\n\nHere's why it works. Hot Dogs, Chili dogs, Italian beef, fries, gyros, bu...",223
7,tp,"2.5 stars? I'm struggling with this place. I wonder if it's because I've built up the excitement about trying it out and am now sorely disappointed? \n\nWe made reservations for my parents' trip into town. A few friends of ours have been here for dinner and raved about it so we figured it would be a fun dinner option. We were seated upstairs which was nice. There were a surprising amount of babies in th

##### Most confident - true negatives

In [122]:
most_confident_predictions(preds, "tn")

,category,text,num_words
0,tn,Food and drinks are really great. \n\nThe music is just hands down annoying.,13
1,tn,"Food is delicious, portions are large and the service is excellent. This is my favorite place to stop when I am spending time in Tampa.",25
2,tn,We decided to go with my family to this restaurant and Julio was so friendly to us. The food was excellent and the service was great for a really good price.,31
3,tn,Very nice experience sitting at the bar at Olivia Saturday night. Service was fantastic and food was great. Will definitely be back,22
4,tn,The food was delicious and atmosphere perfect. Great choice for wine lovers too! Mmmmmm...,14
5,tn,"service was terrific, food was high quality, expect to pay for it.",12
6,tn,Very nice spot to enjoy food made with love. The staff is quick to fill orders & are extremely polite.,20
7,tn,Never had a bad meal at skipjacks. Food is excellent. Staff is friendly. Atmosphere is awesome,16
8,tn,"Excellent atmosphere. Family oriented. Live music, great sangria, superb menu.",10
9,tn,Fantastic food. They provided a gluten-free dinner that was one the best I have ever had.,16


##### Most confident - false positives

In [123]:
most_confident_predictions(preds, "fp")

,category,text,num_words
0,fp,"When you think of sushi and Japanese food, Philly does not come to mind. But if you're stuck here, like I am, then you got to get your fix somewhere. Located near the edge of Chinatown is Yakitori Boy. I have been here several times purely for the karaoke, but I recently had the chance to try some of their food and was pleasantly surprised.\n\nFor those who have never been, the top floor is a karaoke bar with some private rooms in the back. It resembles a bar/lounge and is usually pretty des...",569
1,fp,"fried is my middle name. ok, it's not but it makes more sense when i'm reviewing fried chicken.. \n\nannyhow, this was our first stop after an 8-hour drive from san antonio, texas. having woken up at godforsaken hour of 4am, we were just in time for lunch at willie mae's. as our first ever visit to nola, i wanted our first meal to be extra special and the reviews made the fried chicken sound outstanding. \n\nwe arrived and as a few other reviews mention, this part of town, north of the FQ wa...",394
2,fp,"The Stillery has the distinct honor of being the first restaurant I visited in my inaugural trip to Nashville. Part of me is disappointed that Stillery set the bar so high; once you reach the top, is there any way to go but down? But that would be like resenting Jordan or Gretzky for being the GOAT... all you can do is sit back, admire their work, and appreciate that you got the opportunity to experience their greatness.\n\nMy friend and I sat at the bar and were cordially welcomed by Michae...",318
3,fp,"I am on the beginning of the Indy food truck trail. My quest to try them all and give my honest to goodness, or badness as the case may be, opinion. \nAfter visiting Duos I am very happy to see they have followed 2 great paths in food preparation! First being cleanliness!!! Oh boy that's a big one. I can't even count the times I've walked away from establishments because of that. Duos is CLEAN! The servers in the truck are well groomed (nails and hands) and the truck and preparation area has...",277
4,fp,"Calle Tepa is a refreshing central Tucson stop for not-your-standard Mexican fare. I certainly hope this place sticks around, as it occupies the same location as several former short-term ventures (across from the Park Place Mall). \n The restaurant is tastefully decorated with wall murals. Eating locations include the back bar area, a patio that receives a good mix of afternoon sun and shade, and a family dining area in the front lit with natural light.\n Now for the FOOD:\nThis is like fan...",237
5,fp,"I just want to preface this review with the acknowledgement that I knew what I was getting into. It's PaNeRa BrEAd, I get it, it's bland, it's microwaved. That's it's whole personal brand, it's what the company was built off of-""Bland and boring food to get the job done fast and cheap!"" is the actual company slogan. But normally, when I'm in a rush on my lunch break, I can rely on PB to come through with a solid lunch. But not today apparently! This $17(!!!!-$17?!? For LUNCH?!?!) disaster st...",198
6,fp,"Yes please! \n\nI am a sucker for poki and apparently missed the first location, but was very excited to see a Hoki Poki right on the convenient trip between home and work. Stopped in and was thrilled. \n\nYou start by ordering off of an IPad type system. Pretty standard: select the items you want in your bowl and swipe your card. Easy. Done! While I do enjoy talking to staff, this concept fits well for this location. We were greeted when we entered and they did say to let them know if we ha...",332
7,fp,"When you got that craving for a really, REALLY good, greasy heart-attack of a burger, this is the place in Tucson you just have to hit up. When you hear the tots calling you, you know where you can find them. When you get that strange feeling from somewhere in the depths of your stomach that makes you wonder, ""What would peanut butter on a burger taste like?""....the answer is 'freakin 

##### Most confident - false negatives

In [124]:
most_confident_predictions(preds, "fn")

,category,text,num_words
0,fn,The food was so good! The service was fast and the staff is very friendly! Cannot wait to go back!,20
1,fn,So friendly and quick with traditional Chinese food. Try the sizzling rice soup!,13
2,fn,We've gone a few times. Service is always lacking. Food is never great.,13
3,fn,"The sushi was old and rubbery, the hibachi grill was over seasoned. The buffet was cold and flavorless.",18
4,fn,Strawberry spinach salad - omg! One of the best salads I've ever had.\n\nEverything else there was also quite amazing - definitely a hidden gem!,25
5,fn,I've ate here multiple times but usually forget to take pictures LOL here's some the food there is really good.,20
6,fn,Came hoping for Hattie B's style Nashville hot chicken and they delivered! Best fried chicken in StL.,17
7,fn,"The food is supposed to be good here...I wouldn't know. We sat for 20 minutes in a half full restaurant without a waiter talking to us, so we left",29
8,fn,"My new favorite pizza place, forget about big chain craps pizza. Good service and very good quality ingredients.",18
9,fn,"GET IN MY BELLY! \n\nIt's cot damned delicious here, I don't care what the yuppie transplants say.",17


### Amazon small model, 2 layers finetuned on Yelp

In [ ]:
pretraining_model = "amazon_finetune"  # Meaning "small"
finetuning_dataset = "yelp"
n_layers = 2

model_fp = os.path.join(MODEL_DIR, f"{pretraining_model}_{n_layers}_LAYERS_FINETUNED_ON_{finetuning_dataset}", "results")
timestamps = [f for f in os.listdir(model_fp) if f.startswith("2022")]
assert len(timestamps) == 1
results_fp = os.path.join(model_fp, timestamps[0])
preds_path = os.path.join(results_fp, "predictions.csv")
preds = pd.read_csv(preds_path, encoding="utf-8")
test_set = datasets[finetuning_dataset]["test"][["text", "label"]]
preds["text"] = test_set["text"]
preds["label"] = test_set["label"]
preds["num_words"] = [len(t.split()) for t in preds["text"]]
preds.tail()

##### Accuracy

In [ ]:
accuracy(preds)

##### Classification report

In [ ]:
with open(os.path.join(results_fp, "classification_report.txt")) as report_file:
    print(report_file.read())

##### Confusion matrix - raw & normalized

In [ ]:
confusion_matrix(preds, False)

In [ ]:
confusion_matrix(preds, True)

##### Word count comparison

In [ ]:
preds.groupby(["category"])["num_words"].mean()

##### Most confident - true positives

In [ ]:
most_confident_predictions(preds, "tp")

##### Most confident - true negatives

In [ ]:
most_confident_predictions(preds, "tn")

##### Most confident - false positives

In [ ]:
most_confident_predictions(preds, "fp")

##### Most confident - false negatives

In [ ]:
most_confident_predictions(preds, "fn")

### Amazon small model, 4 layers finetuned on Yelp

In [125]:
pretraining_model = "amazon_finetune"  # Meaning "small"
finetuning_dataset = "yelp"
n_layers = 4

model_fp = os.path.join(MODEL_DIR, f"{pretraining_model}_{n_layers}_LAYERS_FINETUNED_ON_{finetuning_dataset}", "results")
timestamps = [f for f in os.listdir(model_fp) if f.startswith("2022")]
assert len(timestamps) == 1
results_fp = os.path.join(model_fp, timestamps[0])
preds_path = os.path.join(results_fp, "predictions.csv")
preds = pd.read_csv(preds_path, encoding="utf-8")
test_set = datasets[finetuning_dataset]["test"][["text", "label"]]
preds["text"] = test_set["text"]
preds["label"] = test_set["label"]
preds["num_words"] = [len(t.split()) for t in preds["text"]]
preds.tail()

,pred_prob_0,pred_prob_1,yhat,y,category,text,label,num_words
5889,-1.266121,1.783918,1,1,tp,"After months of construction, Yoshi has finally opened. At this point, my anticipation was running so high they could be serving neatly wrapped stones and I'd probably give it a good review. What they do serve, however, is frozen yogurt and sushi. \n\nThe frozen yogurt is the same as in most other asian style frozen yogurt places. They have the basic tart flavor and many others. Yogurt plus toppings is 49 cents/ounce so expect to pay around $4-5 for a decent portion. I thought their topping ...",1,269
5890,0.929161,-1.074188,0,0,tn,This is our favorite restaurant. I always get the lobster ravioli and we usually start with risotto balls. We usually try something different each time we go and loved everything we had from them. The cocktails are great too. They also have the best service around. Everyone is so kind and professional. They're the best.,0,55
5891,-0.141642,0.031598,1,0,fp,"My boyfriend and I went to the drunken crab for our first time tonight. We've never been to a place with this style of eating, so we were super grateful when our waitress came over and explained the menu to us. We didn't want to get super messy so she was very accommodating and gave us all the sauces we wanted on the side for dipping. The king crab was a dream come true, and the calamari was the perfect combo of chewy and crispy. We are huge fans of unique beers and were very happy with the ...",0,106
5892,-0.696703,0.812309,1,1,tp,"So I found out from one of our Uber drivers that most restaurants don't want to serve boiled crawfish because ""they don't want to deal with it"" which is understandable. We came to NOLA during peak crawfish season and were on the hunt. Being from the Bay Area (CA), we are used to the Vietnamese style crawfish so it's literally swimming in either oil or butter. I honestly don't like it because it gives me the worst heartburn and you have to rely on the sauce for flavor. Big EZ Seafood doesn't ...",1,194
5893,0.856756,-1.116879,0,0,tn,I'm from the tri-state area and this pizza is the closest that I've found (in the Franklin area) to the pizza I grew up with! Highly recommend!!,0,27


##### Accuracy

In [126]:
accuracy(preds)

0.7573803868340685

##### Classification report

In [127]:
with open(os.path.join(results_fp, "classification_report.txt")) as report_file:
    print(report_file.read())

              precision    recall  f1-score   support

           0       0.77      0.75      0.76      2992
           1       0.75      0.76      0.76      2902

    accuracy                           0.76      5894
   macro avg       0.76      0.76      0.76      5894
weighted avg       0.76      0.76      0.76      5894




##### Confusion matrix - raw & normalized

In [128]:
confusion_matrix(preds, False)

,Pred = False,Pred = True
,,
Actual = False,2244,748
Actual = True,682,2220


In [129]:
confusion_matrix(preds, True)

,Pred = False,Pred = True
,,
Actual = False,0.380726,0.126909
Actual = True,0.115711,0.376654


##### Word count comparison

In [130]:
preds.groupby(["category"])["num_words"].mean()

category
fn     73.470674
fp    165.279412
tn     51.122995
tp    238.195946
Name: num_words, dtype: float64

##### Most confident - true positives

In [131]:
most_confident_predictions(preds, "tp")

,category,text,num_words
0,tp,"Ya...\n\nSo last time I reviewed Bar Ferdinand which is conveniently adjacent from this establishment ""El Camino Real"" which by far is the closest thing to genuine tasty Tex/Mex and Mex made by ""White Trendy Hipsters"". But to the point I take back what I said about Bar Ferdinand. Especially since I friend the actual artist who did all the promotional art for the place and you can see his work every Thursday on the back of the City Paper advertising both BF and this place. \n\nTheir pulled po...",256
1,tp,"Ella's American Folk Art Café is where all of the hip kids hang out to eat, drink, and soak up the sunshine on their big deck. Ella's feels like a very fun Bohemian gathering spot where you can enjoy some great drinks and terrific meals, amid a lot of interesting people and fascinating artwork. Unfortunately I'll never be able to get a job at Ella's since I don't have enough ink or nose jewelry, but that's ok since I can always show up to enjoy a great meal with some fun people.\n\nElla's lo...",418
2,tp,"Getting here is a feather in my cap. I told myself I'd stroll through the Treme a bit and walking from the French Quarter to this bulletproof glassed liquor store turned Asian restaurant made sure I accomplished this goal. Honestly, I've been through worse neighborhoods so I'd argue that the edge of Treme is not too bad.\n\nThe spot itself was quite the sight. It was basically a liquor store with what looked like a bank teller window at the front. But instead of bank tellers there were some ...",432
3,tp,"I arrived in Conshy for a project a few months ago and immediately went on the coffee hunt. After trying a few places that will not be named where the coffee could be a substitute for rat poison, I was directed to 'feine. \n\nFrom my first step through the door I could tell that this was the type of coffee shop up my alley. Matcha, almond milk, chai, and all the other buzzwords you want to see on a well crafted chalk board in a coffee shop. The owner is passionate about her coffee select...",234
4,tp,"Saengerbund Oktoberfest is an annual festival that celebrates German-American heritage with food, music, beer, amusements and shopping. It occurs the third weekend of September normally. Although it is nothing like Oktoberfests in Germany, it is about as close as you'll get without boarding a plane and using your passport.\n\nWe went and had a great time last Fall. It was an easy trip from the Philadelphia area and offered much more than the weak Octoberfests that seem to pop up and disappea...",370
5,tp,"I have heard mixed reviews about this place and I have not been in years. The one of 96th street closed sometime ago and the only time I had been was for a lunch meeting years ago. Recently, Groupon had a spend $15 deal to get $35 worth of food. I figured with this kind of deal I would give it a try. I got carryout and ordered a few different things to sample. Upon enterning this place in the early evening on the way home from work there was one customer in front of me and that was it. BAD S...",411
6,tp,"""Mostly Loving"" *\n*See below for important caveat\n\nWelcome to vegan fast food -- hardly a novel concept in Philadelphia, a city strong in vegan pursuits and growing stronger by the deep-fried day.\n\nLoving Hut offers the quick-draw Asian version of the trend: spring rolls ""with vegan ham,"" tofu and dampened sweet chili sauce wraps, cans of Coke, something rice, something noodle, and some vegan cake slices I will never try. In terms of quality it's really not very different from airport ...",239
7,tp,"""I'm gonna put on a ton of weight in the next couple of months"" - this is what I told the cashier as I walked out of United Provisions. \n\nAfter months of waiting this place is finally open ! It sure is a blessing for those like me caught right in between the Schnucks on Olive and the Schnucks on Clayton Rd. Having said that, I don't know when my next trip to Schnucks will be, I'm go

##### Most confident - true negatives

In [132]:
most_confident_predictions(preds, "tn")

,category,text,num_words
0,tn,"Service was great, (thanks Luke), food is good, but not great. Milkshakes are thick and delicious.",16
1,tn,"Best Chinese I've found in Nashville. Consistently fresh and tasty! Love their vegetable lo mein, sweet and sour chicken as well we their spring rolls. Always friendly and fast.",29
2,tn,"Went for the first time last Saturday. The brisket was a bit dry, however the pulled pork was pretty tasty. Would not go out of my way to eat here again.",31
3,tn,Ate there today after not going for awhile. It was great! Our server was super nice and accommodating!,18
4,tn,"Never been disappointed! Always friendly staff. Best creme brûlée in Santa Barbara ! I recommend the one with berries, I literally just come here for this.",26
5,tn,Sat outside around 600 and enjoyed watching people go by. Had terrific taquitos for an appetizer and a great chimichanga.,20
6,tn,Great music on the hour. Food was good too. Went on back to back night. Good both times.,18
7,tn,My favorite soup place. Can always try free samples of the soup and it's always fresh and delicious. \n\nMy favorite is the Lobster Bisque and the Wisconsin Cheddar.,28
8,tn,Food was good but not spectacular - better lobster rolls in New England. Clam chowder is a must. Friendly environment.,20
9,tn,Great food. Loved the sourdough rolls. New York Strip was made to perfection. Tasted excellent. Waitress did a great job. Was in the area for work and would highly recommend.,30


##### Most confident - false positives

In [133]:
most_confident_predictions(preds, "fp")

,category,text,num_words
0,fp,"The Location: It is located in Downtown Boise, near the intersection of W Idaho St and S 11th St. Parking looks relatively easy to find.\n\nThe People: Staff and bartenders are good. One of the friendliest bartenders worked at this bar.\n\nThe Drinks:\n- Fernet and Coke: An Argentinian classic drink, if one has not tried it, do so. Bartenders here know how to serve this drink well!\n- Water: It is free, self-serve, stay hydrated.\n\nThe Venue:\n- I came on a Wednesday after checking out Neur...",197
1,fp,"Molly Darcy's was just an ""A-OK"" experience for me. My hubby and I were interested in trying out a new restaurant during lent (for a Friday fish dinner) and since he works in Clayton he suggested we check out MD. When we arrived around 6 pm on a Friday the sign told us to ""seat yourself"" so we did just that. There were a few open tables so it was pretty easy to find something. MD's is not a very large establishment; they have a nice size bar with seating and a few high tops, but we opted for...",510
2,fp,I don't take giving five stars lightly so I will be very clear on the reasons I have chosen to do so. This restaurant came to my attention by my cousin and fellow foodie (CF). CF and I enjoy delicious food from a large variety of restaurants around the city and surrounding area. On an unspecific day last week CF came by with a leftover Spider Bao Bun from Ban Ban. Hours later the the Bun remained full of flavor and delicious. If you are a fan of soft Shell Crab they honor it at this res...,242
3,fp,"The Stillery has the distinct honor of being the first restaurant I visited in my inaugural trip to Nashville. Part of me is disappointed that Stillery set the bar so high; once you reach the top, is there any way to go but down? But that would be like resenting Jordan or Gretzky for being the GOAT... all you can do is sit back, admire their work, and appreciate that you got the opportunity to experience their greatness.\n\nMy friend and I sat at the bar and were cordially welcomed by Michae...",318
4,fp,"Disclaimer: This review is only for the drinks. \n\nFor those familiar with Chicago, the Libertine is the love child of the Whistler and Gilt Bar. Yes, the decor and staff scream hipster galore, but the customers demographics appeared to be more on the preppy end of the spectrum. We stopped by here after dinner for a couple of drinks. I ordered a bourbon cocktail from their menu (Lonely in a Limo) but it had egg whites in it, which I've never had before - I've seen this ingredient appear ...",217
5,fp,"Hidden gem in South Jersey! \n\nAfter almost 20 years of traveling near and far to get this Vietnamese specialty for our ""noodle boy"" son ... we're pretty particular about our pho. So, when he suggested that we try out this place in Deptford, NJ ... our expectations weren't too high, especially as we haven't had much luck previously as we got farther and farther away from our usual places in Philly. \n\nGot there as they opened at 11am on a Saturday and had our pick of tables ... which wer...",284
6,fp,"Corner Pub Midtown, or ""CP2"" as we like to call it (because of it's sister bar, the original Corner Pub in Green Hills) is a great place to meet after work for happy hour, to watch any sporting event, or just to go for a great hot turkey sandwich. \n\nIt's not a stuffy, ultra smoky bar--windows all around, and when the weather is nice they open them fully so it's like the entire bar is a patio. Multiple TVs so anywhere you sit has a good view. Large, non-cramped bar. Cold beer. Acoustic...",132
7,fp,"Second visit to this particular location. Party of two on a Thursday at 9pm, waited just a few minutes from check in until seating. Not too crowded, not too noisy, although there was a particular party with a crying baby.\n\nYour typical TdB outing - salad bar, sides, rolls, plates and plates of meat. I was not at all wowed, however, this time around. The meats were pretty bland, and especially the

##### Most confident - false negatives

In [134]:
most_confident_predictions(preds, "fn")

,category,text,num_words
0,fn,"They park outside of the Tulane hospital. Tried them one time, the chicken platter was great. The sauce on top was tasty. Worth checking out.",25
1,fn,Everything was fabulous. Definitely ate way too much for just two people but every bite was worth it. The boar burger was absolutely fantastic probably one of the best burgers I've ever had and the Brussel sprouts were top notch. If you are in the area you should give Cerdo a try!,52
2,fn,They delivered food today at nursing home in Bensalem with a cockroach in it. Department of Health is informed as well. We have the pictures too.,26
3,fn,The Cleveland Street Roll was delicious! It was huge and fed both of us easily. We also got the philly roll (normal/large size) and the ginger salad had real ginger on top. Yum!,33
4,fn,Coming from a Mexican.. Lol this place is legit. They cook the food fresh on the spot for you. Carne asada is a must .,25
5,fn,Came hoping for Hattie B's style Nashville hot chicken and they delivered! Best fried chicken in StL.,17
6,fn,"Attended old Friends birthday party in side room, food by Brother John's. Pork sliders, beans, collard greens, pretty good. Would return for a full meal.",25
7,fn,One of the best places to get a good burger. Their asada fries are pretty amazing too. It does take a long time but worth the wait. Not the cheapest menu out there but it's good food so I'm always willing to pay,43
8,fn,"A bit expensive for Eggs.\n\nHowever, it was good, fresh and enjoyable.\n\nExpect to pay 20 per person.",18
9,fn,We've gone a few times. Service is always lacking. Food is never great.,13


### Amazon large model, 2 layers finetuned on Yelp

In [ ]:
pretraining_model = "amazon_finetune_LARGE"
finetuning_dataset = "yelp"
n_layers = 2

model_fp = os.path.join(MODEL_DIR, f"{pretraining_model}_{n_layers}_LAYERS_FINETUNED_ON_{finetuning_dataset}", "results")
timestamps = [f for f in os.listdir(model_fp) if f.startswith("2022")]
assert len(timestamps) == 1
results_fp = os.path.join(model_fp, timestamps[0])
preds_path = os.path.join(results_fp, "predictions.csv")
preds = pd.read_csv(preds_path, encoding="utf-8")
test_set = datasets[finetuning_dataset]["test"][["text", "label"]]
preds["text"] = test_set["text"]
preds["label"] = test_set["label"]
preds["num_words"] = [len(t.split()) for t in preds["text"]]
preds.tail()

##### Accuracy

In [ ]:
accuracy(preds)

##### Classification report

In [ ]:
with open(os.path.join(results_fp, "classification_report.txt")) as report_file:
    print(report_file.read())

##### Confusion matrix - raw & normalized

In [ ]:
confusion_matrix(preds, False)

In [ ]:
confusion_matrix(preds, True)

##### Word count comparison

In [ ]:
preds.groupby(["category"])["num_words"].mean()

##### Most confident - true positives

In [ ]:
most_confident_predictions(preds, "tp")

##### Most confident - true negatives

In [ ]:
most_confident_predictions(preds, "tn")

##### Most confident - false positives

In [ ]:
most_confident_predictions(preds, "fp")

##### Most confident - false negatives

In [ ]:
most_confident_predictions(preds, "fn")

### Amazon large model, 4 layers finetuned on Yelp

In [ ]:
pretraining_model = "amazon_finetune_LARGE"
finetuning_dataset = "yelp"
n_layers = 4

model_fp = os.path.join(MODEL_DIR, f"{pretraining_model}_{n_layers}_LAYERS_FINETUNED_ON_{finetuning_dataset}", "results")
timestamps = [f for f in os.listdir(model_fp) if f.startswith("2022")]
assert len(timestamps) == 1
results_fp = os.path.join(model_fp, timestamps[0])
preds_path = os.path.join(results_fp, "predictions.csv")
preds = pd.read_csv(preds_path, encoding="utf-8")
test_set = datasets[finetuning_dataset]["test"][["text", "label"]]
preds["text"] = test_set["text"]
preds["label"] = test_set["label"]
preds["num_words"] = [len(t.split()) for t in preds["text"]]
preds.tail()

##### Accuracy

In [ ]:
accuracy(preds)

##### Classification report

In [ ]:
with open(os.path.join(results_fp, "classification_report.txt")) as report_file:
    print(report_file.read())

##### Confusion matrix - raw & normalized

In [ ]:
confusion_matrix(preds, False)

In [ ]:
confusion_matrix(preds, True)

##### Word count comparison

In [ ]:
preds.groupby(["category"])["num_words"].mean()

##### Most confident - true positives

In [ ]:
most_confident_predictions(preds, "tp")

##### Most confident - true negatives

In [ ]:
most_confident_predictions(preds, "tn")

##### Most confident - false positives

In [ ]:
most_confident_predictions(preds, "fp")

##### Most confident - false negatives

In [ ]:
most_confident_predictions(preds, "fn")